<h1 style="text-align:center;">Secret Sharing for Secure Distributed Matrix Multiplication</h1>

## 1. Introduction

Matrix multiplication is a fundamental operation in mathematics that is used in various areas and applications such as machine learning, neural networks, image processing, optimization and others. In many of these cases, we come across situations in which the matrices to be multiplied have high dimensions, so it becomes costly to compute this multiplication locally.

As a result, an interest arises in computing matrix multiplication in a distributed way, i.e., with the help of servers to optimize the process. However, the entries in the matrices may contain private information and the servers may be curious, i.e., any one of them may try to deduce some information about the matrices. Thus, an interest arises in the problem of how to optimize matrix multiplication in a secure and distributed way, known as **Secure Distributed Matrix Multiplication** (SDMM).

The problem was first introduced in 2018 by Chang et.al in [1]. The statement consists of a user who wants to compute the product $AB$ of two matrices $A$ and $B$ with the help of $N$ servers in such a way that no server is able to deduce any information about the matrices and the total communication cost is minimized. An interesting approach to this problem is that we can use Secret Sharing to create PIR schemes. The main idea is to use polynomials over finite fields, since polynomials provide a simple and efficient way of generating and combining secret data such as solving linear systems or taking a polynomial interpolation. In addition, finite fields will provide algebraic and statistical properties that make SDMM schemes perfectly secure from the Information Theory perspective.

In this notebook, we will present Secure Distributed Matrix Multiplication using Secret Sharing schemes, motivating from the most straightforward scheme to the family of GASP (Gap Arithmetic Secure Polynomial) codes introduced in [2, 3]. For all possible solutions to the problem, we will present a possible implementation of the algorithm and a demonstration of why the scheme has the properties of computing the multiplication while keeping all the matrix entries secure.

## 2. Secure Distributed Matrix Multiplication

In Secure Distributed Matrix Multiplication (SDMM), a user wants to compute multiplication $AB$ of two matrices $A \in \text{GF}^{m\times n}(p)$ and $B \in \text{GF}^{n\times l}(p)$ with the help of $N$ servers where any $T$ of them collude, where $\text{GF}(p)$ is finite field with $p$ elements. The user's goals are to optimize the computation of the product $AB$ in such a way that no server is able to deduce any information about the entries in $A$ and $B$. 

In the scenario in question, the user queries the servers to compute expensive tasks and the servers respond with solutions that the user can use to completely determine the desired result $AB$. The main idea is to define two polynomials $f(x)$ and $g(x)$ so that when interpolating the polynomial $h(x)=f(x)\cdot g(x)$, the product $AB$ can be computed. Let us illustrate this process with a simple example.

**Example 1:** Assume that there are $N=3$ where they do not collude, i.e., they do not communicate with each other to try to find out any information about the matrices, but they can try to find out information by themselves. Also, suppose a user that has two matrices $A \in \text{GF}^{m\times n}(p)$ and $B \in \text{GF}^{n\times l}(p)$, and wants to compute the product $AB \in \text{GF}^{m\times l}(p)$. A simple way to solve this problem is through a generalization of what we know as Secret Sharing [4,5], as described below:

- **Step 1:** The user defines two discrete random variables $R \in \text{GF}^{m\times n}(p)$ and $S \in \text{GF}^{n\times l}(p)$ uniformly at random in their respective spaces.

- **Step 2:** The user defines two polynomials $f(x)=A+Rx$ and $g(x)=B+Sx$ over the finite field $\text{GF}(p)$.

- **Step 3:** The user sends two evaluations $f(i)$ and $g(i)$ of each polynomial to each server $i \in \{1,2,3\}$. $(3<p)$

- **Step 4:** Each server $i$ computes the multiplication $h(i)=f(i)\cdot g(i)=AB+(AS+RB)i+RSi^2$ and sends to the user.

- **Step 5:** The user interpolates the polynomial $h(x)$ using the three evaluations $h(i)$ to compute the $AB$.

The user can decode $AB$ by the fact that it has three evaluations of the degree three polynomial $h(x)$, so any three different points completely determine the coefficients of $h(x)$, like $AB$. We give an demonstration of how this decoding process can be done in Python.

We use an the libraries called "galois" and NumPy. The library galois provides many interesting things for implementing codes using finite fields arithmetic, learning about finite field theory (for example, the non-prime finite fields), provide error correction codes, etc. We can install galois from PyPI running "pip install galois" in the terminal. The galois library also offers perfect integration with the NumPy library, which provides functions for performing operations with arrays, polynomial interpolations, solving linear systems, etc. (NumPy can be installed by running "pip install numpy" in the terminal).

So, suppose two matrices $A \in \text{GF}^{2\times 3}(5)$ and $B \in \text{GF}^{2\times 4}(5)$ defined by
\begin{align*}
A = \begin{bmatrix}
1 & 2 & 1 \\
4 & 1 & 2
\end{bmatrix}
\ \text{ and  } \
B = \begin{bmatrix}
1 & 3 \\
2 & 1 \\
1 & 3
\end{bmatrix}.
\end{align*}

Thus, this matrices can be defined as follows:

In [35]:
import galois
import numpy as np

GF5 = galois.GF(5) # defines the finite field with 5 elements GF(5)={0,1,2,3,4}

A = GF5(np.array([[1, 2, 1], [4, 1, 2]]))
B = GF5(np.array([[1, 3], [2, 1], [1, 3]]))

print(A)
print(B)

[[1 2 1]
 [4 1 2]]
[[1 3]
 [2 1]
 [1 3]]


Now, we choose the variables $R \in \text{GF}^{2\times 3}(5)$ and $S \in \text{GF}^{3\times 2}(5)$ uniformly at random in their respective spaces.

In [36]:
# The command GF.Random((n,m)) defines a random matrix of dimensions n by m in the finite field GF

R = GF5.Random((2,3))
S = GF5.Random((3,2))

print(R)
print(S)

[[0 3 1]
 [1 1 4]]
[[1 0]
 [3 1]
 [3 0]]


Next, we define the polynomial evaluations that will be sent to each server:

In [37]:
f1 = A + R * GF5(1)
g1 = B + S * GF5(1)
print('The evaluation f(1) is')
print(f1)
print('The evaluation g(1) is')
print(g1)
print('')

f2 = A + R * GF5(2)
g2 = B + S * GF5(2)
print('The evaluation f(2) is')
print(f2)
print('The evaluation g(2) is')
print(g2)
print('')

f3 = A + R * GF5(3)
g3 = B + S * GF5(3)
print('The evaluation f(3) is')
print(f3)
print('The evaluation g(3) is')
print(g3)

The evaluation f(1) is
[[1 0 2]
 [0 2 1]]
The evaluation g(1) is
[[2 3]
 [0 2]
 [4 3]]

The evaluation f(2) is
[[1 3 3]
 [1 3 0]]
The evaluation g(2) is
[[3 3]
 [3 3]
 [2 3]]

The evaluation f(3) is
[[1 1 4]
 [2 4 4]]
The evaluation g(3) is
[[4 3]
 [1 4]
 [0 3]]


Each server $i \in \{1,2,3\}$ compute the product $h(i)=f(i) \cdot g(i)$ and send to the user:

In [38]:
h1 = f1 @ g1
print('The answer h(1) is')
print(h1)
print('')

h2 = f2 @ g2
print('The answer h(2) is')
print(h2)
print('')

h3 = f3 @ g3
print('The answer h(3) is')
print(h3)

The answer h(1) is
[[0 4]
 [4 2]]

The answer h(2) is
[[3 1]
 [2 2]]

The answer h(3) is
[[0 4]
 [2 4]]


Finally, since $h(x)=f(x)\cdot g(x)=AB+(AS+RB)x+RSx^2$, the variables that we need to determine in order to interpolate $h(x)$ are $AB$, $AS+RB$ and $RS$. So, if we denote
\begin{align*}
AB = \begin{bmatrix}
a & b\\
c & d
\end{bmatrix}
,\ \
AS+RB = \begin{bmatrix}
e & f \\
g & h
\end{bmatrix}
\ \text{ and  } \
RS = \begin{bmatrix}
p & q \\
r & s
\end{bmatrix},
\end{align*}
we can note that 
\begin{align*}
h(x) = \begin{bmatrix}
a_1 & b_1\\
c_1 & d_1
\end{bmatrix}
+
\begin{bmatrix}
a_2 & b_2 \\
c_2 & d_2
\end{bmatrix}
x+\begin{bmatrix}
a_3 & b_3 \\
c_3 & d_3
\end{bmatrix}x^2=
\begin{bmatrix}
a_1 & b_1\\
c_1 & d_1
\end{bmatrix}
+
\begin{bmatrix}
a_2x & b_2x \\
c_2x & d_2x
\end{bmatrix}
x+\begin{bmatrix}
a_3x^2 & b_3x^2 \\
c_3x^2 & d_3x^2
\end{bmatrix}
\doteq
\begin{bmatrix}
h_{1}(x) & h_2(x) \\
h_3(x) & h_4(x)
\end{bmatrix}
.
\end{align*}
 
In other words, through evaluations
\begin{align*}
h(i) = 
\begin{bmatrix}
h_a(i) & h_b(i) \\
h_c(i) & h_d(i)
\end{bmatrix}
,
\end{align*}
we need to interpolate the four polynomials
\begin{align*}
&h_a(x) = a_1+a_2x+a_3x^2, h_b(x) = b_1+b_2x+b_3x^2, \\
&h_c(x) = c_1+c_2x+c_3x^2, h_d(x) = d_1+d_2x+d_3x^2,
\end{align*}
to determine $AB$.

Therefore, since we send evaluations $h(i)$ for each server $i \in \{1,2,3\}$, we have, for $j \in \{a,b,c,d\}$, the following system
\begin{align*}
    \begin{bmatrix}
    1^0 & 1^1 & 1^2\\
    2^0 & 2^1 & 2^2\\
    3^0 & 3^1 & 3^2
    \end{bmatrix}
    \begin{bmatrix}
    j_1 \\
    j_2 \\
    j_3
    \end{bmatrix}
    =
    \begin{bmatrix}
    1 & 1 & 1\\
    1 & 2 & 4\\
    1 & 3 & 4
    \end{bmatrix}
    \begin{bmatrix}
    j_1 \\
    j_2 \\
    j_3
    \end{bmatrix}
    =
    \begin{bmatrix}
    h_j(1) \\
    h_j(2) \\
    h_j(3)
    \end{bmatrix}
\end{align*}
to solve in $\text{GF}(5)$, where the $3 \times 3$ matrix is Vandermonde matrix with coefficients $1,2,3 \in \text{GF}(5)$ denoted by $V(1,2,3)$. Since the Vandermonde matrix is always invertible for different nonzero coefficients in any finite field $\text{GF}(p)$, the system always has a unique solution. With this in mind, let us present a code to determine the product $AB$:

In [39]:
Vandermonde = GF5(np.array([[1, 1, 1], [1, 2, 4], [1, 3, 4]]))
AB = GF5(np.array([[0, 0], [0, 0]]))
for i in range(2):
    for j in range(2):
        y = GF5(np.array([h1[i,j], h2[i,j], h3[i,j]]))
        sol = np.linalg.solve(Vandermonde, y)
        AB[i, j] = sol[0]

print('The product AB is')
print(AB)

The product AB is
[[1 3]
 [3 4]]


Thus, the result of the multiplication $AB$ in $\text{GF}(5)$ is 
\begin{align*}
    AB = \begin{bmatrix}
    1 & 2 & 1 \\
    4 & 1 & 2
    \end{bmatrix}
    \cdot
    \begin{bmatrix}
    1 & 3 \\
    2 & 1 \\
    1 & 3
    \end{bmatrix}
    =
    \begin{bmatrix}
    1 & 3 \\
    3 & 4 
    \end{bmatrix}
    \in \text{GF}^{2\times 2}(5).
\end{align*}

The security of the scheme follows directly from the Security property of Secret Sharing in which the fact that we choose the variables $R \in \text{GF}^{2\times 3}(5)$ and $S \in \text{GF}^{3\times 2}(5)$ uniformly at random in their respective spaces guarantees that the evaluations $f(i)$ and $g(i)$ are uniformly at random and statistically independent of the values of the entries $A$ or $B$. In other words, we have that
\begin{align*}
    \text{Pr}[A=a|f(i)=r]=\text{Pr}[A=a]
\end{align*}
for any $a,r \in \text{GF}^{2\times 3}(5)$ and
\begin{align*}
    \text{Pr}[B=b|g(i)=s]=\text{Pr}[B=b]
\end{align*}
for any $b,s \in \text{GF}^{3\times 2}(5)$.

Thus, the fact that each server owns shares $f(i)$ and $g(i)$ is irrelevant, as this fact has no influence on the probability of the server obtaining any information about $A$ or $B$.

**Remark 1:** We have a Jupyter Notebook that teaches Secret Sharing in great detail at this link https://github.com/IgorAureliano/git-SSJupyter/blob/main/SSjupyterlab.ipynb. If the reader of this notebook doesn't know or has any doubts about Secret Sharing, we strongly recommend reading this notebook.

In the previous process, note that each server performs a multiplication of matrices of the same dimensions as the original product. More specifically, each server does the work of multiplying $f(i) \in \text{GF}^{2\times 3}(5)$ and $g(i) \in \text{GF}^{3\times 2}(5)$ to send back to the user $h(i) \in \text{GF}^{2\times 2}(5)$, i.e., each server does the same job if we multiply $A \in \text{GF}^{2\times 3}(5)$ and $B \in \text{GF}^{3\times 2}(5)$ to get $AB \in \text{GF}^{2\times 2}(5)$. 

In this example, the process is not so costly, since we are dealing with matrices of small dimensions, but suppose that the matrices have arbitrarily large dimensions. In that case, the processes of evaluating the polynomials $f(x)$ and $g(x)$, uploading the evaluations to each server, performing the multiplication $h(i)=f(i)\cdot g(i)$, downloading the answers from the servers and decoding the matrix $AB$ could be so computationally expensive that it would be faster for the user to compute the product locally without the help of the servers.

The conclusion is that, depending on the conditions of the matrix, the direct application of the “generalized” version of Secret Sharing may not be such an efficient way to solve the problem of determining the multiplication $AB$. So, the following question arises: “Is there a more efficient way of computing this product in such a way that the sum of all computational processes will be faster than the user computing the matrix locally?”

Many SDMM works, such as [1]-[6], consider the idea of partitioning the matrices $A$ and $B$ in blocks for certain parameters $K$ and $L$ as below:
\begin{align*}
    A = 
\begin{bmatrix}
A_1 \\
A_2 \\
\vdots \\
A_K
\end{bmatrix},
\quad
B = 
\begin{bmatrix}
B_1 & B_2 & \dots & B_L
\end{bmatrix} 
\implies 
AB =
\begin{bmatrix}
    A_1 B_1  &  \dots  &  A_1 B_L    \\
    \vdots  &  \ddots  &  \vdots     \\
    A_K B_1  &  \dots  &  A_K B_L
\end{bmatrix}
\end{align*}

Thus, computing the product $AB$ is equivalent to computing all the submatrices $A_kB_l$.

The idea proceeds in the same way as the previous example, a user defines polynomials $f(x)$ and $g(x)$ which carry information about, respectively, $A$ and $B$ to send to the servers. The servers multiply $f(x)$ and $g(x)$ which results in the evaluation of a new polynomial $h(x)$ which is sent back to the user. The user must be able to compute the multiplication $AB$ through all the evaluations.

This schematization is interesting, because, it can be proven that for certain partitions, the process can be faster than the user computing the product locally, but we have to be careful. Let us illustrate the problems that we can have with this strategy with an example.

**Example 2:** Let $A \in \text{GF}^{6\times 2 }(p)$ and $B \in \text{GF}^{2\times 6}(p)$ and a partition defined for $K=L=3$ as follows:
\begin{align*}
    A=
    \begin{bmatrix}
        A_1 \\
        A_2 \\
        A_3
    \end{bmatrix}
    ,\ \ \ \ 
    B=
    \begin{bmatrix}
        B_1 & B_2 & B_3
    \end{bmatrix}
    \implies AB=
    \begin{bmatrix}
        A_1B_1 & A_1B_2 & A_1B_3\\
        A_2B_1 & A_2B_2 & A_2B_3\\
        A_3B_1 & A_3B_2 & A_3B_3
    \end{bmatrix}
    .
\end{align*}

Furthermore, suppose that there are $N$ servers where any $T=2$ can collude. In this case, following the intuition of the previous example, we could define the following polynomials:
\begin{align*}
    &f(x)=A_1+A_2x+A_3x^2+R_1x^3+R_2x^4,\\
    &g(x)=B_1+B_2x+B_3x^{2}+S_1x^{3}+S_2x^{4},
\end{align*}
where we need $T=2$ random variables $R_1,R_2 \in \text{GF}^{6\times 2}$ to add randomness to the informations of $A$ and another $T=2$ random variables $S_1,S_2 \in \text{GF}^{2\times 6}$ to add randomness to the Informations of $B$.

However, when we multiply these two polynomials, we get the following polynomial
\begin{align*}
    h(x)&=A_1B_1+(A_1B_2+A_2B_1)x+(A_1B_3+A_3B_1)x^2+...+R_2S_2x^8.
\end{align*}

Note that when we interpolate this polynomial, we will be able to determine the submatrix $A_1B_1$, however $A_1B_2$ and $A_2B_1$ will not be possible to determine, because we will be determining the value of (A_1B_2+A_2B_1) and not the submatrices separately. The same problem occurs with the term that $(A_1B_3+A_3B_1)$ follows $x^2$. This directly implies the fact that we cannot choose any polynomials $f(x)$ and $g(x)$.

So, consider the following definition for the two associated polynomials:
\begin{align*}
    &f(x)=A_1+A_2x+A_3x^2+R_1x^3+R_2x^4,\\
    &g(x)=B_1+B_2x^5+B_3x^{10}+S_1x^{13}+S_2x^{14}.
\end{align*}

This scheme is a curious, because note that $g(x)$ has some gaps in the degrees. But, we can multiply $f(x)$ and $g(x)$ to see that 
\begin{align*}
    h(x)&=A_1B_1+A_2B_1x+A_3B_1x^2+R_1B_1x^3+R_2B_1x^4+A_1B_2x^5+A_2B_2x^6+A_3B_2x^7\\
    &+R_1B_2x^8+R_2B_2x^9+A_1B_3x^{10}+A_2B_3x^{11}+A_3B_3x^{12}+(R_1B_3+A_1S_1)x^{13}\\
    &+(A_2R_1+A_1S_2+R_2B_3)x^{14}+(A_3R_1+A_2S_2)x^{15}+(R_1S_1+A_3S_2)x^{16}\\
    &+(R_2S_1+R_1S_2)x^{17}+R_2S_2x^{18}.
\end{align*}

In this case, note that all the submatrices $A_kB_l$ appear alone as coefficients of $h(x)$. Therefore, as the degree of $h(x)$ is equal to 18, if we have $18+1=19$ different evaluations of $h(i)$, we will be able to interpolate $h(x)$ to determine all its coefficients, which implies that we will be able to compute the product $AB$. In addition, the fact that the scheme is secure follows directly from Secret Sharing's security property.

One way of looking at this problem was to minimize the degree of $h(x)$. In this way, we would have a less costly process of evaluating the associated polynomials and decoding them, since we would need fewer servers to compute the product $AB$. But, consider the following example:
\begin{align*}
    &f_*(x)=A_1+A_2x+A_3x^2+R_1x^{9}+R_2x^{12},\\
    &g_*(x)=B_1+B_2x^3+B_3x^{6}+S_1x^{9}+S_2x^{10}.
\end{align*}

If we multiply these two associated polynomials, we get the following polynomial:
\begin{align*}
    h_*(x)&=A_1B_1+A_2B_1x+A_3B_1x^2+A_1B_2x^3+A_2B_2x^4+A_3B_2x^5+A_1B_3x^6+A_2B_3x^7\\
    &+A_3B_3x^8+(A_1S_1+R_1B_1)x^9+(A_2S_1+A_1S_2)x^{10}+(A_2S_2+A_3S_1)x^{11}+(A_3S_2+R_2B_1)x^{12}\\
    &+(R_1B_3+R_2B_2)x^{15}+R_1S_1x^{18}+R_1S_2x^{19}+R_2S_1x^{21}+R_2S_2x^{22}.
\end{align*}

These two polynomials provide the peculiarity that the degree of $h_*(x)$ is 22, but $h_*(x)$ does not have the terms $x^{13},x^{14},x^{16},x^{17},x^{20}$. This is equivalent to concluding that the coefficients accompanying these terms in $h(x)$ are a null matrix. Thus, we only need to determine 17 variables in $h(x)$, which implies that we need only $N=17+1=18$ servers. The conclusion is that in this case we would need fewer servers to carry out the process, because $18<19$.

A common metric for evaluating the efficiency of such schemes is the download rate. The download rate $\mathcal{R}$ is defined as the number of symbols of the desired information $AB$ by the total amount of symbols that the user downloads to obtain the answers from the servers, i.e.,
\begin{align*}
    \mathcal{R}=\frac{\text{desired symbols}}{\text{symbols downloaded}}.
\end{align*}

If the matrices are $A \in \text{GF}^{m\times n}(p)$ and $B \in \text{GF}^{n\times l}(p)$, the submatrices will be $A_i \in \text{GF}^{\frac{m}{K}\times n}(p)$ for $i \in [K]$ and $B_j \in \text{GF}^{n\times \frac{l}{L}}(p)$ for $j \in [L]$. Thus, the download rate is given by
\begin{align*}
    \mathcal{R}=\frac{K\cdot L \cdot \frac{m}{K} \cdot \frac{l}{L}}{N\cdot \frac{m}{K} \cdot \frac{l}{L}}=\frac{K\cdot L}{N}.
\end{align*}

In our example $(K=L=3)$, the associated polynomials
\begin{align*}
    &f(x)=A_1+A_2x+A_3x^2+R_1x^3+R_2x^4,\\
    &g(x)=B_1+B_2x^5+B_3x^{10}+S_1x^{13}+S_2x^{14}.
\end{align*}
would define a scheme that needs $N=19$ servers. Therefore, the download rate would be
\begin{align*}
    \mathcal{R}=\frac{3\cdot 3}{19}=\frac{9}{19}.
\end{align*}

On the other hand, the associated polynomials
\begin{align*}
    &f_*(x)=A_1+A_2x+A_3x^2+R_1x^{9}+R_2x^{12},\\
    &g_*(x)=B_1+B_2x^3+B_3x^{6}+S_1x^{9}+S_2x^{10}.
\end{align*}
would define a scheme that needs $N=18$ servers. Therefore, the download rate would be
\begin{align*}
    \mathcal{R}=\frac{3\cdot 3}{18}=\frac{1}{2}.
\end{align*}

This is an improvement on the efficiency of the previous scheme, since $\frac{1}{2}<\frac{9}{19}$. In this way, this example leads us to conclude that it is not because the degree of the polynomials are higher that we will have a lower efficiency.

Since the polynomial $h_*(x)$ have some gaps in the degrees, let us revisit the properties of Decodability and Security.

Define the set of exponents which occur in the the polynomial h(x) by
\begin{align*}
    \mathcal{J} = \{0, . . . , 8, 9, 10, 11, 12, 15, 18, 19, 21, 22\}.
\end{align*}

The Decodability property is based on finding a evaluation vector $\mathbf{a}=[a_1,a_2,...,a_{18}]^t \in \text{GF}^{18}(p)$ such that the Generalized Vandermonde matrix defined by 
\begin{align*}
    GV(\mathbf{a}, \mathcal{J})=
    \begin{bmatrix}
        a_i^j
    \end{bmatrix}
    , \ 1\leq i \leq 18, \ j \in \mathcal{J},
\end{align*}
is invertible.

One could say that it is enough to choose the vector $\mathbf{a}=[1,2,...,18]^t \in \text{GF}^{18}(p)$ as we have been doing so far, but we should be a little careful. For example, we can have two numbers $a_1,a_2 \in \text{GF}(p)$ such that
\begin{align*}
    &X = R_1a_1^9+R_2a_1^{12} \\
    &Y = R_1a_2^9+R_2a_2^{12}
\end{align*}
are linearly dependent. Then, the two servers that has $f(a_1)$ and $f(a_2)$ can obtain information about the matrix $A$.

One numeric example can be $a_1=4$ and $a_2=7$ in $\text{GF}(31)$. Then, 
\begin{align*}
    &f(4) = A_1+4A_2+16A_3+8R_1+16R_2, \\
    &f(8) = A_1+7A_2+18A_3+8R_1+16R_2.
\end{align*}
Which implies that
\begin{align*}
    f(7)-f(4)=7A_2+2A_3,
\end{align*}
i.e., the two servers extra information about a combination of the two blocks $A_2$ and $A_3$.

This can happen because there is a lemma that that says: Let $p$ be a prime number, $q=p^n$ for some $n \in \mathbb{Z}_+$ and $f:\text{GF}(q) \rightarrow \text{GF}(q)$ such that $x \mapsto x^j$. Then, if $gcd(j,q-1)=1$, $f$ is bijective. The contrapositive causes the situation that we must be careful of, because the scheme will not possess the Security property in this case.

Therefore, the Security property will be satisfied provided that the matrices
\begin{align*}
    P_{n,m}=
    \begin{bmatrix}
        a_n^{9} & a_n^{12}\\
        a_m^{9} & a_m^{12}
    \end{bmatrix}
    \ \ \ 
    Q_{n,m}=
    \begin{bmatrix}
        a_n^{9} & a_n^{10}\\
        a_m^{9} & a_m^{10}
    \end{bmatrix}
\end{align*}
are invertible for $1\leq n\neq m \leq 18$, i.e., the pairs of combinations 
\begin{align*}
    &X_n = R_1a_n^9+R_2a_n^{12} \\
    &X_m = R_1a_m^9+R_2a_m^{12}
\end{align*}
and 
\begin{align*}
    &Y_n = R_1a_n^9+R_2a_n^{10} \\
    &Y_m = R_1a_m^9+R_2a_m^{10}
\end{align*}
are linearly independent.

Note that for this to happen, we need that
\begin{align*}
    \det(P_{n,m})=a_n^9a_m^9(a_m^3-a_n^3)\neq 0 \text{ and } \det(Q_{n,m})=a_n^9a_m^9(a_m-a_n)\neq 0.
\end{align*}

Therefore, it suffices that the conditions $a_n^3\neq a_m^3$ and $a_n\neq a_m$ are satisfied to guarantee the Security property. The lemma guides us to choose a finite field $\text{GF}(p)$ such that $gcd(3,q-1)$ as $\text{GF}(23)$ or $\text{GF}(29)$ and $a_n\neq a_m$ guide us to choose any entry in the evaluation vector $\mathbf{a}$ different.

The conclusion is that the proofs of the Decodability and Security properties are guaranteed if, respectively, the matrices $GV(\mathbf{a},\mathcal{J})$, $P_{n,m}$ and $Q_{n,m}$ are invertible.

In general, observe that given $K$ and $L$ the number of partitions of, respectively, the matrices $A$ and $B$, the problem consists of choosing associated polynomials $f(x)$ and $g(x)$ such that the number of servers $N$ needed to interpolate $h(x)=f(x)g(x)$ is minimized. Thus, the problem in last example could be presented as follows: 

If we denote a set $\{1,2,...,n\} \subseteq \mathbb{N}$ by $[n]$, given polynomials  
\begin{align*}
    &f(x)=A_1x^{\alpha_1}+A_2x^{\alpha_2}+A_3x^{\alpha_3}+R_4x^{\alpha_4}+R_5x^{\alpha_5},\\
    &g(x)=B_1x^{\beta_1}+B_2x^{\beta_2}+B_3x^{\beta_3}+S_4x^{\beta_4}+S_5x^{\beta_5},
\end{align*}
determine the exponents $\alpha_*$ and $\beta_*$ such that
\begin{align*}
    \alpha_k+\beta_l \neq \alpha_{k'}+\beta_{l'},\ \forall (k,l) \in [3] \times [3],\ \forall (k',l') \in [5] \times [5],\ (k,l) \neq (k',l'),
\end{align*}
and the number of terms to be determined in
\begin{align*}
    h(x)&=\sum_{1\leq k,l\leq 3} A_kB_l x^{\alpha_k+\beta_l} + \sum_{\substack{1\leq k\leq 3 \\ 4\leq l\leq 5}} A_kS_l x^{\alpha_k+\beta_l}\\
    &=\sum_{\substack{4\leq k\leq 5 \\ 1\leq l\leq 3}} R_kB_l x^{\alpha_k+\beta_l} + \sum_{4\leq k,l\leq 5} R_kS_l x^{\alpha_k+\beta_l}
\end{align*}
is minimized. Furthermore, determine a evaluation vector $\mathbf{a}=[a_1,a_2,...,a_{18}]^t \in \text{GF}^{18}(p)$ such that $P_{n,m}$ and $Q_{n,m}$ are invertible.

The problem of determining the $\alpha_k$ and $\beta_l$ can be seen in an alternative way through the following addition table:
$$
\begin{aligned}
\begin{array}{|c|ccc|cc|}
\hline
+ & \beta_1 & \beta_2 & \beta_3 & \beta_4 & \beta_5 \\
\hline
\alpha_1 & \alpha_1+\beta_1 & \alpha_1+\beta_2 & \alpha_1+\beta_3 & \alpha_1+\beta_4 & \alpha_1+\beta_5 \\
\alpha_2 & \alpha_2+\beta_1 & \alpha_2+\beta_2 & \alpha_2+\beta_3 & \alpha_2+\beta_4 & \alpha_2+\beta_5 \\
\alpha_3 & \alpha_3+\beta_1 & \alpha_3+\beta_2 & \alpha_3+\beta_3 & \alpha_3+\beta_4 & \alpha_3+\beta_5 \\
\hline
\alpha_4 & \alpha_4+\beta_1 & \alpha_4+\beta_2 & \alpha_4+\beta_3 & \alpha_4+\beta_4 & \alpha_4+\beta_5 \\
\alpha_5 & \alpha_5+\beta_1 & \alpha_5+\beta_2 & \alpha_5+\beta_3 & \alpha_5+\beta_4 & \alpha_5+\beta_5 \\
\hline
\end{array}
\end{aligned}
$$

This table is called **Degree Table**. The name comes from the fact that each entry that appears in this table tells us the terms of each degree that appear in the polynomial $h(x)$. Note that each repeated term occurs in the table in the same way as in the polynomial $h(x)$ and implies that we have a combination of coefficient matrices accompanied by a repeated degree term. For example, the Degree Table for the associated polynomials
\begin{align*}
    f(x)=A_1+A_2x+A_3x^2+R_1x^{9}+R_2x^{12} \text{ and }g(x)=B_1+B_2x^3+B_3x^{6}+S_1x^{9}+S_2x^{10}
\end{align*}
is
$$
\begin{aligned}
\begin{array}{|c|ccc|cc|}
\hline
+ & \beta_1=0 & \beta_2=3 & \beta_3=6 & \beta_4=9 & \beta_5=10 \\
\hline
\alpha_1=0 & 0 & 3 & 6 & 9 & 10 \\
\alpha_2=1 & 1 & 4 & 7 & 10 & 11 \\
\alpha_3=2 & 2 & 5 & 8 & 11 & 12 \\
\hline
\alpha_4=9 & 9 & 12 & 15 & 18 & 19 \\
\alpha_5=12 & 12 & 15 & 18 & 21 & 22 \\
\hline
\end{array}
\end{aligned}
$$

Note that the number of different terms that appear in the table is 18 and they are in the set
\begin{align*}
    \mathcal{J} = \{0, . . . , 8, 9, 10, 11, 12, 15, 18, 19, 21, 22\}
\end{align*}
of the degrees that appers in $h(x)=f(x)\cdot g(x)$.

The upper-left block in the table describes the terms that accompany the variables $A_kB_l$, so they must be distinct in order to avoid the problem of the other scheme where there is a combination of desired variables accompanied by a term of a specific degree. 

In addition, the necessary condition to have the Security property is to choose $\alpha_4 \neq \alpha_5$ and $\beta_4 \neq \beta_5$, because this will ensure that
\begin{align*}
    \det(P_{n,m})\neq 0 \text{ and } \det(Q_{n,m})\neq 0
\end{align*}
for any vector $\mathbf{a}=[a_1,a_2,...,a_{18}]^t \in \text{GF}^{18}(29)$ with each entry distinct and non-zero.


Before we define some notions for studying the Degree Table and some general schemes, let us give an example of how we can implement the scheme with associated polynomials 
\begin{align*}
    &f(x)=A_1+A_2x+A_3x^2+R_1x^{9}+R_2x^{12},\\
    &g(x)=B_1+B_2x^3+B_3x^{6}+S_1x^{9}+S_2x^{10},
\end{align*}
over the finite field $\text{GF}(29)$.

For this implementation, consider $A \in \text{GF}^{6\times 2 }(29)$ and $B \in \text{GF}^{2\times 6}(29)$ as follows:
\begin{align*}
    A =
    \begin{bmatrix}
        1 & 2 \\
        3 & 4 \\
        5 & 6 \\
        7 & 8 \\
        9 & 10 \\
        11 & 12
    \end{bmatrix}
    ,\ \ \ \ 
    B =
    \begin{bmatrix}
        1 & 28 & 2 & 27 & 3 & 26 \\
        10 & 20 & 11 & 19 & 12 & 18
    \end{bmatrix}.
\end{align*}
Thus, the partitions
\begin{align*}
    A=
    \begin{bmatrix}
        A_1 \\
        A_2 \\
        A_3
    \end{bmatrix}
    ,\ \ \ \ 
    B=
    \begin{bmatrix}
        B_1 & B_2 & B_3
    \end{bmatrix}
    ,
\end{align*}
are such that
\begin{align*}
    &A_1= \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}, \quad A_2= \begin{bmatrix} 5 & 6 \\ 7 & 8 \end{bmatrix}, \quad A_3= \begin{bmatrix} 9 & 10 \\ 11 & 12 \end{bmatrix}, \\
    &B_1= \begin{bmatrix} 1 & 28 \\ 10 & 20 \end{bmatrix}, \quad B_2= \begin{bmatrix} 2 & 27 \\ 11 & 19 \end{bmatrix}, \quad B_3= \begin{bmatrix} 3 & 26 \\ 12 & 18 \end{bmatrix}.
\end{align*}

So, the first thing to do is to define these partitions

In [40]:
GF29 = galois.GF(29)

A1 = GF29(np.array([[1,2], [3,4]]))
A2 = GF29(np.array([[5,6], [7,8]]))
A3 = GF29(np.array([[9,10], [11,12]]))
B1 = GF29(np.array([[1,28], [10,20]]))
B2 = GF29(np.array([[2,27], [11,19]]))
B3 = GF29(np.array([[3,26], [12,18]]))

and the random variables $R_1,R_2 \in \text{GF}^{2\times 2 }(29)$ and $S_1,S_3 \in \text{GF}^{2\times 2}(29)$ uniformly at random.

In [41]:
R1, R2 = GF29.Random((2,2)), GF29.Random((2,2))
S1, S2 = GF29.Random((2,2)), GF29.Random((2,2))

Now, we compute and define the set of all evaluations of the associated polynomials $f(x)$ and $g(x)$. So, since $N=18$, we have

In [42]:
f_evaluations = {}
g_evaluations = {}

for i in range(1,19):
    f_evaluations[i] = A1 + A2 * GF29(i) + A3 * GF29(i)**2 + R1 * GF29(i)**9 + R2 * GF29(i)**(12)
    g_evaluations[i] = B1 + B2 * GF29(i)**3 + B3 * GF29(i)**6 + S1 * GF29(i)**9 + S2 * GF29(i)**(10)

With all the evaluations computed, the user sends to the servers to compute the following set of evaluations of the polynomial $h(x)=f(x)\cdot g(x)$:

In [43]:
h_evaluations = {}

for i in range(1,19):
    h_evaluations[i] = f_evaluations[i] @ g_evaluations[i]

Next, we define a list the of exponents which occur in the the polynomial h(x), i.e., a list with the elements of the set
\begin{align*}
    \mathcal{J} = \{0, . . . , 8, 9, 10, 11, 12, 15, 18, 19, 21, 22\}.
\end{align*}

In [44]:
J = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 18, 19, 21, 22]

So, the Generalized Vandermonde matrix $GV(\mathbf{a}, \mathcal{J})=\begin{bmatrix}
        a_i^j
    \end{bmatrix}$ for $\mathbf{a}=[a_1,a_2,...,a_{18}]^t=[1,2,...,18]^t \in \text{GF}^{18}(29)$, $i \in \{1,2,...,18\}$ and $j \in \mathcal{J}$ is given by

In [45]:
a = GF29(np.array([i for i in range(1,19)]))

GV = GF29(np.array([[ai ** j for j in J] for ai in a]))

A quick verification guarantees that $GV(\mathbf{a}, \mathcal{J})$ is invertible, since its determinant is non-zero, as we discussed.

In [46]:
print(np.linalg.det(GV))

20


Thus, $h(x)$ can be interpolated using the evaluation vector $\mathbf{a}=[1,2,...,18]^t \in \text{GF}^{18}(29)$.

Finally, we present a possible code to determine each submatrix $A_kB_l$ and decode $AB$:

In [47]:
from itertools import product

all_submatrices = {}
for l in range(1,4):
    for k in range(1,4):
        all_submatrices[f'A_{k}B_{l}'] = GF29(np.zeros((2, 2), dtype=int))

for i in range(2):
    for j in range(2):
        y = []
        for m in range(1,19):
            y.append(h_evaluations[m][i,j])
        y = GF29(np.array(y))
        sol = np.linalg.solve(GV, y)
        
        elementos = list(range(1, 4))
        list_of_index = list(product(elementos, repeat=2)) # create a list of all ordered index pairs (k,l) to acess the submatrices A_kB_l in 'all_submatrices'
        for f in range(9):
            all_submatrices[f'A_{list_of_index[f][1]}B_{list_of_index[f][0]}'][i,j] = sol[f]

AB = np.block([
    [all_submatrices[f'A_{1}B_{1}'], all_submatrices[f'A_{1}B_{2}'], all_submatrices[f'A_{1}B_{3}']],
    [all_submatrices[f'A_{2}B_{1}'], all_submatrices[f'A_{2}B_{2}'], all_submatrices[f'A_{2}B_{3}']],
    [all_submatrices[f'A_{3}B_{1}'], all_submatrices[f'A_{3}B_{2}'], all_submatrices[f'A_{3}B_{3}']]
])

print("The multiplication AB is given by")
print(AB)

The multiplication AB is given by
[[21 10 24  7 27  4]
 [14 19 21 12 28  5]
 [ 7 28 18 17  0  6]
 [ 0  8 15 22  1  7]
 [22 17 12 27  2  8]
 [15 26  9  3  3  9]]


We can verify that this is exactly the product $AB$ by defining the matrices and computing locally:

In [48]:
A = GF29([[1, 2],[3, 4],[5, 6],[7, 8],[9, 10],[11, 12]])
B = GF29([[1, 28, 2, 27, 3, 26],[10, 20, 11, 19, 12, 18]])
print(A @ B)

[[21 10 24  7 27  4]
 [14 19 21 12 28  5]
 [ 7 28 18 17  0  6]
 [ 0  8 15 22  1  7]
 [22 17 12 27  2  8]
 [15 26  9  3  3  9]]


## 5. Polynomial Codes

**Definition 1:** Let $n be \in \mathbb{Z}_+$. Then the set of all integers from 1 to $n$ is denoted by
\begin{align*}
    [n]=\{1,2,...,n\}\subseteq \mathbb{Z}_+.
\end{align*}

Again, we set the configuration where a user wants to compute the product of two matrices $A \in \text{GF}^{m\times n}(p)$ and $B \in \text{GF}^{n\times l}(p)$. The user partition $A$ and $B$ into, respectively, $K \in \mathbb{Z}_+$ and $L\in \mathbb{Z}_+$ blocks/submatrices such that 
\begin{align*}
    A=
    \begin{bmatrix}
        A_1\\
        \vdots\\
        A_K
    \end{bmatrix}
    \text{ and }
    B=
    \begin{bmatrix}
        B_1 & \cdots & B_L
    \end{bmatrix}
    ,
\end{align*}
where $A_i \in \text{GF}^{\frac{m}{K}\times n}_q$ for $i \in [K]$ and $B_j \in \text{GF}^{n\times \frac{l}{L}}(p)$ for $j \in [L]$.

Thus, the product $AB=[A_kB_l]_{1\leq k \leq K,1\leq l \leq L}$, i.e.,
\begin{align*}
    AB=
    \begin{bmatrix}
        A_1B_1 & \cdots & A_1B_L\\
        \vdots & \ddots & \vdots\\
        A_KB_1 & \cdots & A_KB_L
    \end{bmatrix}
    .
\end{align*}

Thus, we formally define a polynomial code:

**Definition 2:** Let $K,L,T,N$ be positive integers such that $T<N$, $\alpha=[\alpha_1,...,\alpha_{K+T}]^t \in \mathbb{N}^{K+T}$ and $\beta=[\beta_1,...,\beta_{L+T}]^t \in \mathbb{N}^{L+T}$, then a **Polynomial Code** $\text{PC}(K,L,T,N,\alpha,\beta)$ is a tool used to compute the product $AB$ as follows:

- **Step 1:** Choose T matrices $R_1,R_2,...,R_T \in \text{GF}^{\frac{m}{K}\times n}(p)$ and T matrices $S_1,S_2,...,S_T \in \text{GF}^{n\times \frac{l}{L}}(p)$ uniformly at random in their respective space.
- **Step 2:** Define the polynomials 
\begin{align*}
    &f(x)=\sum_{k=1}^K A_kx^{\alpha_k}+\sum_{t=1}^T R_tx^{\alpha_{K+t}},\\
    &g(x)=\sum_{l=1}^L B_lx^{\beta_l}+\sum_{t=1}^T S_tx^{\beta_{L+t}},
\end{align*}
and 
\begin{align*}
    h(x)= f(x)g(x).
\end{align*}
- **Step 3:** If $N$ servers is enough to interpolate $h(x)$, the user choose a evaluation vector $\mathbf{a}= [a_1,...,a_N]^t \in \text{GF}^N(p)$.
- **Step 4:** For each server $i$ send the evaluations $f(a_i)$ and $g(a_i)$ for the server computes the product $h(a_i)=f(a_i)g(a_i)$ and transmits it back to the user.

- **Step 5:** The user then interpolates the polynomial $h(x)$ given all of the evaluations $h(a_i)$ and attempts to recover all products $A_kB_l$ from the coefficients of $h(x)$ to decode $AB$.

**Definition 3** A polynomial code $\text{PC}(K,L,T,N,\alpha,\beta)$ is **Decodable** and **T-Secure** if there is some evaluation vector $\mathbf{a}=[a_1,a_2,...,a_{N}]^t \in \text{GF}^{N}(p)$ such that the following conditions hold:
1. **(Decodabilty)** All the products $A_kB_l$ for $k = 1,...,K$ and $l=1,...,L$ are completely determined by the evaluations $h(an)$ for $i=1,...,N$, i.e., for any $y \in \text{GF}^{m \times l}(p)$ and $x_1,x_2,...,x_N \in \text{GF}^{\frac{m}{K}\times \frac{l}{L}}(p)$, we have
\begin{align*}
    \text{Pr}[AB=y|h(a_1)=x_1,...,h(a_N)=x_N]=1.
\end{align*}
2. **($T$-Security)** For any set $\{i_1,i_2,...,i_T\} \subset [N]$, $y \in \text{GF}^{m \times n}(p)$, $z \in \text{GF}^{n \times l}(p)$, $y_1,y_2,...,y_T \in \text{GF}^{\frac{m}{K}\times n}(p)$ and $z_1,z_2,...,z_T \in \text{GF}^{n\times \frac{l}{L}}(p)$, we have 
\begin{align*}
    \text{Pr}[A=y,B=z|f(a_{i_1})=y_1,g(a_{i_1})=z_1,...,f(a_{i_T})=y_T,g(a_{i_T})=z_T]=\text{Pr}[A=y,B=z].
\end{align*}

**Definition 4:** Let a polynomial code $\text{PC}(K,L,T,N,\alpha,\beta)$ be decodable and $T$-secure. Then, the download rate of $\text{PC}(K,L,T,N,\alpha,\beta)$ is 
\begin{align*}
    \mathcal{R}=\frac{KL}{N}.
\end{align*}

This definition follows from the fact that the coefficients $A_kB_l$ are the same size as the evaluations $h(a_i)$, the user downloads $N$ answers, $k=1,...,K$ and $l=1,...,L$.

## 6. Degree Table


**Definition 5:** Let $\alpha=[\alpha_1,...,\alpha_{K+T}]^t \in \mathbb{N}^{K+T}$ and $\beta=[\beta_1,...,\beta_{L+T}]^t \in \mathbb{N}^{L+T}$. Then, the **Outer Sum** $\alpha \oplus \beta \in \mathbb{N}^{(K+T)\times (L+T)}$ of $\alpha$ and $\beta$ is defined to be the matrix
\begin{align*}
    \alpha \oplus \beta=
    \begin{bmatrix}
        \alpha_1+\beta_1 & \cdots & \alpha_1+\beta_{L+T}\\
        \vdots & \ddots & \vdots\\
        \alpha_{K+T}+\beta_1 & \cdots & \alpha_{K+T}+\beta_{L+T}
    \end{bmatrix}
    .
\end{align*}

**Definition 6:** Let $\alpha=[\alpha_1,...,\alpha_{K+T}]^t \in \mathbb{N}^{K+T}$ and $\beta=[\beta_1,...,\beta_{L+T}]^t \in \mathbb{N}^{L+T}$. Then, we say that the outer sum $\alpha \oplus \beta$ is decodable and $T$-secure if the following conditions are satisfied:
- (Decodability) $(\alpha \oplus \beta)_{k,l} \neq (\alpha \oplus \beta)_{k',l'}$ for all $\forall (k,l) \in [K] \times [L]$ and all $(k',l') \in [K+T] \times [L+T]$.
- ($T$-security) $\alpha_{K+t}\neq \alpha_{K+t'}$ and $\beta_{L+t}\neq \beta_{L+t'}$ for any $t \neq t' \in [T]$.

One way to better understand the above definitions is to look at the table below:

$$
\begin{aligned}
\begin{array}{|c|ccc|ccc|}
\hline
+                                                      & \beta_1                & \cdots                 & \beta_L                & \beta_{L+1} & \cdots & \beta_{L+T} \\ \hline
\alpha_1                                               & \alpha_1+\beta_1 & \cdots & \alpha_1+\beta_L & \alpha_1+\beta_{L+1} &  \cdots  & \alpha_1+\beta_{L+T} \\ 
\vdots                                                 & \vdots & \ddots & \vdots &   \vdots  &    \ddots  & \vdots \\ 
\alpha_K                                               & \alpha_K+\beta_{1} & \cdots & \alpha_K+\beta_{L} &   \alpha_K+\beta_{L+1} &    \cdots  &  \alpha_K+\beta_{L+T} \\ \hline
\alpha_{K+1}                   & \alpha_{K+1}+\beta_{1} &  \cdots  &  \alpha_{K+1}+\beta_{L} & \alpha_{K+1}+\beta_{L+1} & \cdots & \alpha_{K+1}+\beta_{L+T} \\ 
\vdots                         & \vdots  & \ddots &  \vdots  & \vdots  &   \ddots  & \vdots \\ 
\alpha_{K+T} & \alpha_{K+T}+\beta_{1} &  \cdots  &  \alpha_{K+T}+\beta_{L} & \alpha_{K+T}+\beta_{L+1} &  \cdots & \alpha_{K+T}+\beta_{L+T} \\ \hline
\end{array}
\end{aligned}
$$

The Decodability property states that each $\alpha_k+\beta_l$ in the upper-left block with entries $\alpha_k+\beta_l$ for all $(k,l) \in [K]\times [L]$ must be distinct from every other entry in the degree table $\alpha \oplus \beta$. The condition of $T$-security states that all $\alpha_{K+t}$ for all $t \in [T]$ must be pairwise distinct and all $\beta_{L+t}$ for all $t \in [T]$ must be pairwise distinct.

**Definition 7:** Let $A \in \mathbb{N}^{m\times n}$ be a matrix. The **Terms** of $A$ is the set
\begin{align*}
    \text{terms}(A)=\{n \in \mathbb{N}: A[i,j]=n\}
\end{align*}
where $A[i,j]$ denote the entry of $A$ in row $i$ and column $j$.

**Lemma 1:** Let a polynomial code $\text{PC}(K,L,T,N,\alpha,\beta)$ with associated polynomials
\begin{align*}
        &f(x)=\sum_{k=1}^K A_kx^{\alpha_k}+\sum_{t=1}^T R_tx^{\alpha_{K+t}},\\
        &g(x)=\sum_{l=1}^L B_lx^{\beta_l}+\sum_{t=1}^T S_tx^{\beta_{L+t}}
    \end{align*}
where $R_1,R_2,...,R_T \in \mathbb{F}^{\frac{m}{K}\times n}_q$ and $S_1,S_2,...,S_T \in \mathbb{F}^{\frac{n}{L}\times p}_q$ are uniformly at random in their respective space.
    
Then, 
\begin{align*}
    h(x)=f(x)g(x)=\sum_{j \in \mathcal{J}}C_jx^j 
\end{align*}
for some matrices $C_j$ and $\mathcal{J}=\text{terms}(\alpha \oplus \beta)$.

**Proof:** The $C_j$ matrices are defined by multiplication and depend on the numerical values of the degrees
\begin{align*}
    \alpha_k+\beta_l \text{ for }1\leq k\leq K+T \text{ and }1\leq l\leq L+T
\end{align*}
that appear in $h(x)$, i.e., that appear in $\text{terms}(\alpha \oplus \beta)$.

Finally, $\text{terms}(\alpha \oplus \beta)$ describes all the degrees that appear in $h(x)$ and the repetitions imply a grouping of terms in the multiplication. Thus, $C_j$ and $\mathcal{J}=\text{terms}(\alpha \oplus \beta)$ completely define
\begin{align*}
    h(x)=f(x)g(x)=\sum_{j \in \mathcal{J}}C_jx^j .
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$


**Lemma 2:** Let $X_1,X_2,...,X_n \in \text{GF}^{m\times n}(p)$ be uniformly at random. In addition, define
\begin{align*}
    Y_j=a_{j1}X_1+a_{j2}X_2+...+a_{jn}X_n,
\end{align*}
$1\leq j\leq n$ and $a_{j1},a_{j2},...,a_{jT} \in \text{GF}(p)$.

Thus, if $[a_{11},a_{12},...,a_{1n}]^t,...,[a_{n1},a_{n2},...,a_{nn}]^t \in \text{GF}^n(p)$ are linearly independent, the joint probability of $Y_1,Y_2,...,Y_n$ is
\begin{align*}
    \text{Pr}[Y_1=r_1,Y_2=r_2,...,Y_n=r_n]=\text{Pr}[X_1=r_1,X_2=r_2,...,X_n=r_n]=\left(\frac{1}{|\text{GF}^{m\times n}(p)|} \right)^n,
\end{align*}
if $r_1,r_2,...,r_n \in \text{GF}^{m\times n}(p)$.

**Proof:** Initially, as $X_i \in \text{GF}^{m\times n}(p)$ is uniform, it follows that
\begin{align*}
    \text{Pr}[X_i=r_i]=\frac{1}{|\text{GF}^{m\times n}(p)|}.
\end{align*}
    
Thus, since $X_1,X_2,...,X_n$ are uniformly at random, we have that
\begin{align*}
    \text{Pr}[X_1=r_1,X_2=x_2,...,X_T=x_T]=\prod_{i=1}^n\text{Pr}[X_i=x_i]=\left(\frac{1}{|\text{GF}^{m\times n}(p)|} \right)^n.
\end{align*}

Now, note that we can manipulate a realization of the combination $Y_j$ as follows:
\begin{align*}
    &Y_j=r_j\iff a_{j1}X_1+a_{j2}X_2+...+a_{jT}X_T=r_j\\
    &\iff X_i=a_{ji}^{-1}(r_j-a_{j1}X_1-...-a_{jT}X_T)\doteq x_i
\end{align*}
    
In this way, if we are observing the joint probability of $Y_j,X_1,...,X_{j-1},X_{j+1},...,X_T$, the combination
\begin{align*}
    x_i = a_{ji}^{-1}(r_j-a_{j1}X_1-...-a_{jT}X_T)
\end{align*}
is a realization in $\text{GF}^{m\times n}(p)$, because $X_1,...,X_{j-1},X_{j+1},...,X_T$ assume specific values in $\text{GF}^{m\times n}(p)$.
    
Thus, applying this idea for $Y_1,Y_2,...,Y_n$, we can isolate the discrete random variables $X_i$ in each observation to compute the joint probability of $Y_1,Y_2,...,Y_n$ as follows
\begin{align*}
    \text{Pr}[Y_1=r_1,Y_2=r_2,...,Y_n=r_n]=\text{Pr}[X_1=x_1,X_2=x_2,...,X_n=x_n]=\left(\frac{1}{|\text{GF}^{m\times n}(p)|} \right)^n.
\end{align*}

And, since $r_1,r_2,...,r_n \in \text{GF}^{m\times n}(p)$ are arbitrary, we conclude that
\begin{align*}
    \text{Pr}[Y_1=r_1,Y_2=r_2,...,Y_n=r_n]=\text{Pr}[X_1=r_1,X_2=r_2,...,X_n=r_n]=\left(\frac{1}{|\text{GF}^{m\times n}(p)|} \right)^n.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Definition 8:** Let $M \in \mathbb{F}^{m\times n}$ be a matrix. We say that $M$ is a **MDS Matrix** if and only if every square sub-matrix of M is non-singular.

**Lemma 3:** Let $\text{PC}(K,L,T,N,\alpha,\beta)$ be a polynomial code such that $\alpha \oplus \beta$ is decodable and $T$-secure. In addition, suppose that $\mathcal{J}=\text{terms}(\alpha \oplus \beta)$ and there is a evaluation vector $\mathbf{a}=[a_1,...,a_N]^t \in \text{GF}^N(p)$ such that the following properties are satisfied:
1. (Decodability) The Generalized Vandermonde Matrix $GV(\mathbf{a}, \mathcal{J})$ is invertible.
2. ($T$-Security) The $T\times N$ matrices
\begin{align*}
    P=
    \begin{bmatrix}
        a_n^{\alpha_{K+t}}
    \end{bmatrix}
    \text{ and }Q=
    \begin{bmatrix}
        a_n^{\beta_{L+t}}
    \end{bmatrix}
    ,\ 1\leq n \leq N, \  1\leq t \leq T,
\end{align*}
is a MDS matrix.
Then, $\text{PC}(K,L,T,N,\alpha,\beta)$ is decodable and $T$-secure.

**Proof:** First, we prove that $\text{PC}(K,L,T,N,\alpha,\beta)$ is decodable. By **Lemma 1**, since $GV(\mathbf{a}, \mathcal{J})$ is invertible, we can determine all $N$ coefficients $C_j$ of 
\begin{align*}
    \sum_{j \in \mathcal{J}}C_jx^j
\end{align*}
through the $N$ evaluations $h(a_n)$. In addition, since $\alpha \oplus \beta$ is decodable, each coefficient $A_kB_l$ appears alone in $h(x)$, which implies that we can recover all $A_kB_l$ to determine $AB$ completely. Thus, for any $y \in \text{GF}^{m \times l}(p)$ and $x_1,x_2,...,x_N \in \text{GF}^{\frac{m}{K}\times \frac{l}{L}}(p)$, we have
\begin{align*}
    \text{Pr}[AB=y|h(a_1)=x_1,...,h(a_N)=x_N]=1.
\end{align*}

Now, we prove that $\text{PC}(K,L,T,N,\alpha,\beta)$ is $T$-secure. Initially, we state that
\begin{align*}
    &f(x)=\sum_{k=1}^K A_kx^{\alpha_k}+\sum_{t=1}^T R_tx^{\alpha_{K+t}},\\
    &g(x)=\sum_{l=1}^L B_lx^{\beta_l}+\sum_{t=1}^T S_tx^{\beta_{L+t}},
\end{align*}
where $R_1,R_2,...,R_T \in \text{GF}^{\frac{m}{K}\times n}(p)$ and $S_1,S_2,...,S_T \in \text{GF}^{n\times \frac{l}{L}}(p)$ are uniformly at random in their respective space.

In addition, for $j \in \{i_1,i_2,...,i_T\} \subset [N]$, define
\begin{align*}
    &Y_j=\sum_{t=1}^T R_ta_{j}^{\alpha_{K+t}} \text{ and }Z_j=\sum_{t=1}^T S_ta_{j}^{\beta_{L+t}}.
\end{align*}
    
Since, $\alpha \oplus \beta$ is $T$-secure and $P,Q$ is MDS, any group of $T$ rows or columns of $P,Q$ are linearly independent. Thus, we can manipulate the joint distributions of $f(a_{i_1}),f(a_{i_2}),...,f(a_{i_T})$ and $g(a_{i_1}),g(a_{i_2}),$ $...,g(a_{i_T})$ to find the following relation
\begin{align*}
    &\text{Pr}[f(a_{i_1})=r_1,...,f(a_{i_T})=r_T]=\text{Pr}[Y_{i_1}=y_1,...,Y_{i_T}=y_T]\\
    &\text{Pr}[g(a_{i_1})=s_1,...,g(a_{i_T})=s_T]=\text{Pr}[Z_{i_1}=z_1,...,Z_{i_T}=z_T]
\end{align*}
where $r_j,y_j$ and $s_j,z_j$ are random realizations in, respectively, $\text{GF}^{\frac{m}{K}\times n}(p)$ and $\text{GF}^{n\times \frac{l}{L}}(p)$ with the relations
\begin{align*}
    y_j=r_j-\sum_{k=1}^K A_ka_{j}^{\alpha_k} \text{ and }z_j=s_j-\sum_{l=1}^LB_la_{j}^{\beta_l}.
\end{align*}

In this way, by **Lemma 2**, the joint probability of $Y_{i_1},Y_{i_2},...,Y_{i_T}$ and $Z_{i_1},Z_{i_2},...,Z_{i_T}$ are
\begin{align*}
    &\text{Pr}[f(a_{i_1})=r_1,...,f(a_{i_T})=r_T]=\text{Pr}[Y_{i_1}=y_1,...,Y_{i_T}=y_T]=\left(\frac{1}{|\text{GF}^{\frac{m}{K}\times n}(p)|} \right)^T,\\
    &\text{Pr}[g(a_{i_1})=s_1,...,g(a_{i_T})=s_T]=\text{Pr}[Z_{i_1}=z_1,...,Z_{i_T}=z_T]=\left(\frac{1}{|\text{GF}^{n\times \frac{l}{L}}(p)|} \right)^T.
\end{align*}

In addition, to compute the conditional probability of $f(a_{i_1}),f(a_{i_2}),...,f(a_{i_T})$ and $g(a_{i_1}),g(a_{i_2}),...,g(a_{i_T})$ given $A,B$, as $A_1,...,A_K,B_1,...,B_L$ will be fixed, we have the same result
\begin{align*}
    &\text{Pr}[f(a_{i_1})=r_1,...,f(a_{i_T})=r_T|A=c,B=d]=\text{Pr}[Y_{i_1}=y_1,...,Y_{i_T}=y_T]=\left(\frac{1}{|\text{GF}^{\frac{m}{K}\times n}(p)|} \right)^T,\\
    &\text{Pr}[g(a_{i_1})=s_1,...,g(a_{i_T})=s_T|A=c,B=d]=\text{Pr}[Z_{i_1}=z_1,...,Z_{i_T}=z_T]=\left(\frac{1}{|\text{GF}^{n\times \frac{l}{L}}(p)|} \right)^T.
\end{align*}
for ant $c \in \text{GF}^{m\times n}(p)$ and $d \in \text{GF}^{n\times l}(p)$.

So, note that the observation of the evaluations of the polynomial $f(x)$ does not affect the observation of $g(x)$ and vice versa, since $A$ is independent of $B$ and the variables $R_1,R_2,...,R_T,S_1,S_2,...,S_T$ are independent of each other. Thus, if we denote the joint observation $(f(a_{i_1})=r_1,g(a_{i_1})=s_1,...,f(a_{i_T})=r_T,g(a_{i_T})=s_T)$ by an event $E$,
\begin{align*}
    &\text{Pr}[E|A=c,B=d]=\left(\frac{1}{|\text{GF}^{\frac{m}{K}\times n}(p)|} \right)^T\cdot \left(\frac{1}{|\text{GF}^{n\times \frac{l}{L}}(p)|} \right)^T=\text{Pr}[E].
\end{align*}

Then, by Bayes theorem and the previous relations, if we denote the joint event $(f(a_{i_1})=r_1,g(a_{i_1})=s_1,...,f(a_{i_T})=r_T,g(a_{i_T})=s_T)$ by $E$, it follows that
\begin{align*}
    \text{Pr}[A=c,B=d|E]&=\frac{\text{Pr}[E|A=c,B=d]\ \text{Pr}[A=c,B=d]}{\text{Pr}[E]}\\
    &=\frac{\left(\frac{1}{|\text{GF}^{\frac{m}{K}\times n}(p)|} \right)^T \left(\frac{1}{|\text{GF}^{n\times \frac{l}{L}}(p)|} \right)^T\ \text{Pr}[A=c,B=d]}{\left(\frac{1}{|\text{GF}^{\frac{m}{K}\times n}(p)|} \right)^T \left(\frac{1}{|\text{GF}^{n\times \frac{l}{L}}(p)|} \right)^T}\\
    &=\text{Pr}[A=c,B=d].
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Now, we will state some lemmas to prove that there will always be an evaluation vector $\mathbf{a}=[a_1,...,a_N]^t \in \text{GF}^N(p)$ such that the conditions of **Lemma 3** are satisfied.

**Lemma 4:** Let $\mathbf{a}=[a_1,a_2,...,a_n]^t \in \mathbb{Z}^n$ such that $0<a_1<a_2<...<a_n$. In addition, let $\mathcal{J} \subset \mathbb{N}$ such that $|\mathcal{J}|=n$. Then, the Generalized Vandermonde matrix 
\begin{align*}
    GV(\mathbf{a}, \mathcal{J})=
    \begin{bmatrix}
        a_i^k
    \end{bmatrix}
    , \ 1\leq i \leq n, \ k \in \mathcal{J},
\end{align*}
is MDS.

**Proof:** We need to show that given $x_1,...,x_N \in \mathbb{Z}_+$ distinct positive elements such that $\{a_1,...,a_n\} \subseteq \{x_1,...,x_N\}$, then every submatrix of the classical Vandermonde matrix $V(x_1,...,x_N)= \left( x_i^{j-1} \right)_{i,j=1}^{N}$ has a non-zero determinant. Thus, since the Generalized Vandermonde matrix is a submatrix of $V(x_1,...,x_N)$ when
\begin{align*}
    \max_{j \in \mathcal{J}} j = N,
\end{align*}
we will be able to conclude that $GV(\mathbf{a}, \mathcal{J})$ is invertible over $\mathbb{Z}$.

First, observe that any $n \times n$ submatrix of $V(x_1,...,x_N)$ has the form
\begin{align*}
    GV=
    \begin{bmatrix}
        x_{i_l}^{k_j}
    \end{bmatrix}
    ,
\end{align*}
where $j,l \in [N], \{i_1,i_2,...,i_n\} \subseteq [N]$ such that $0<i_1<i_2<...<i_n$ and $\{k_1,k_2,...,k_n\} \subseteq [N-1]$ such that $k_1<k_2<...<k_n$.

So, let $f(x)=c_1x^{k_1}+c_2x^{k_2}+...+c_nx^{k_n}$ be a polynomial. Descartes's rule of signs ensures that the number of positive roots of $f(x)$ does not exceed the number of sign changes among the coefficients $c_1,c_2,...,c_n$, so $f(x)$ has at most $n-1$ positive roots. Therefore, the equation
\begin{align*}
    GV\cdot 
    \begin{bmatrix}
        c_1 \\
        c_2 \\
        \vdots \\
        c_n
    \end{bmatrix}
    =
    \begin{bmatrix}
        0 \\
        0 \\
        \vdots \\
        0
    \end{bmatrix}
\end{align*}
has only the trivial solution $c_1=c_2=...=c_n=0$. This implies that $\text{rank}(GV)=n$, i.e., $GV$ is invertible.

Since $k \in [N]$ is arbitrary, it follows that any submatrix is invertible, i.e., $GV$ is MDS.


$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Lemma 5:** Let $A \in \mathbb{Z}^{n\times n}$ be a matrix such that $\text{det}(A)=x\in \mathbb{Z}$. Then, denoting by $A' \in \text{GF}^{n\times n}(p)$ the matrix obtained by doing the operation $\text{mod} \ p$ to all entries of $A$, we have that $\text{det}(A')=x (\text{mod} \ p) \in \text{GF}(p)$ where $p$ is a prime number.

**Proof:** By definition of the finite field $\text{GF}(p)=\{\overline{0},\overline{1},...,\overline{p-1}\}$, we know that the operations are defined as, for $\overline{a},\overline{b} \in \text{GF}(p)$, we do the usual $\mathbb{Z}$ addition, divide $(a+b)$ by $p$ to get the remainder $0\leq r \leq p-1$ and define $\overline{a+b}=\overline{r}$. We define multiplication in the same way. 

Thus, we have 
\begin{align*}
    \text{det}(A') = \text{det}(A)\ (\text{mod}\ p)= x\ (\text{mod}\ p).
\end{align*}


$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Lemma 6:** Let $A \in \mathbb{Z}^{n\times n}$ be an invertible matrix. Then, when we transform the matrix $A$ to $\text{GF}^{n\times n}(p)$, denoted by $A'$, there is a prime number $p$ such that $A'$ is invertible.

**Proof:** Denote $\text{det}(A)=x$. So, as $A$ is invertible, we have that $\text{det}(A)=x\neq 0$.

Let $p$ be such that $p>x$. In this way, we do not need to apply the modulo operation to turn the integer $x$ into an element of $\text{GF}(p)$ and, by **Lemma 5**, it follows that
\begin{align*}
    \text{det}(A') = x\ (\text{mod}\ p) \neq 0\ (\text{mod}\ p) .
\end{align*}


$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Lemma 7:** Let $\text{PC}(K,L,T,N,\alpha,\beta)$ be a polynomial code such that $\alpha \oplus \beta$ is decodable and $T$-secure. In addition, suppose that $\mathcal{J}=\text{terms}(\alpha \oplus \beta)$. Then, there is a evaluation vector $\mathbf{a}=[a_1,...,a_N]^t \in \text{GF}^N(p)$ such that the following properties are satisfied:
1. (Decodability) The Generalized Vandermonde Matrix $GV(\mathbf{a}, \mathcal{J})$ is invertible.
2. ($T$-Security) The $T\times N$ matrices
\begin{align*}
    P=
    \begin{bmatrix}
        a_n^{\alpha_{K+t}}
    \end{bmatrix}
    \text{ and }Q=
    \begin{bmatrix}
        a_n^{\beta_{L+t}}
    \end{bmatrix}
    ,\ 1\leq n \leq N, \  1\leq t \leq T,
\end{align*}
is a MDS matrix.

**Proof:** Using **Lemma 4**, we know that $GV(\mathbf{a}, \mathcal{J})$ is MDS over $\mathbb{Z}$, i.e., $\text{det}(GV(\mathbf{a}, \mathcal{J}))\neq 0$. In addition, if we denote by $X$ any submatrix of $GV(\mathbf{a}, \mathcal{J})$, by **Lemma 6**, there exists a prime $p'$ for each $X$ such that $\text{det}(X) \ (\text{mod}\ p')\neq 0\ (\text{mod}\ p')$ where we can take $p'$ as the first prime greater than $\text{det}(X)$. 

Thus, denoting by $\mathcal{P}$ the set of all primes $p'$ chosen in such a way for each submatrix of $GV(\mathbf{a}, \mathcal{J})$, it suffices to choose
\begin{align*}
    p=\max_{p' \in \mathcal{P}} p'
\end{align*}
to ensure that $GV(\mathbf{a}, \mathcal{J})$ is MDS, because then the operation $\text{mod}\ p$ will not be performed on the integer $\text{det}(X) \neq 0$, which implies that $\text{det}(X)\ (\text{mod}\ p) \neq 0\ (\text{mod}\ p)$.

Therefore, by the choice of $p$, there will be always an evaluation vector $\mathbf{a}=[a_1,...,a_N]^t \in \text{GF}^N(p)$ such that the properties 1 and 2 are satisfied.

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

So, we have that there will always be an evaluation vector that satisfies the conditions of **Lemma 3** and the construction of a polynomial code for SDMM can be done by choosing $\alpha$ and $\beta$ such that $\alpha \oplus \beta$ is decodable and T-secure. Our objective is to minimize $N=|\text{terms}(\alpha \oplus \beta)|$ subject to $\alpha \oplus \beta$ be decodable and $T$-secure.

## 7. Polynomial codes for big $T$

In this section, we will present a family of polynomial codes called $\text{GASP}_\text{big}$ and in the next section we will present another family of polynomial codes called $\text{GASP}_\text{small}$. We will see later that when $T<\text{min}\{K,L\}$, the $\text{GASP}_\text{small}$ scheme performs better than the $\text{GASP}_\text{big}$ scheme with respect to the goal of minimizing $N=|\text{terms}(\alpha \oplus \beta)|$ subject to $\alpha \oplus \beta$ being decodable and $T$-secure. And, when $T\geq \text{min}\{K,L\}$, the $\text{GASP}_\text{big}$ scheme performs better than the $\text{GASP}_\text{small}$ scheme.

**Definition 9:** Let $K,L,T \in \mathbb{N}^*$. Let $\alpha$ and $\beta$ given by

1. If $L\leq K$:

\begin{align*}
\alpha_k=
\begin{cases}
    k-1,& \text{ if }1\leq k \leq K\\
    KL+t-1,& \text{ if }k=K+T, \ 1\leq t \leq T
\end{cases}
,\ \beta_l=
\begin{cases}
    K(l-1),& \text{ if }1\leq l \leq L\\
    KL+t-1,& \text{ if }l=L+T, \ 1\leq t \leq T
\end{cases}
.
\end{align*}

2. If $L> K$:

\begin{align*}
\alpha_l=
\begin{cases}
    K(l-1),& \text{ if }1\leq l \leq L\\
    KL+t-1,& \text{ if }l=L+T, \ 1\leq t \leq T
\end{cases}
, \ \beta_k=
\begin{cases}
    k-1,& \text{ if }1\leq k \leq K\\
    KL+t-1,& \text{ if }k=K+T, \ 1\leq t \leq T
\end{cases}
.
\end{align*}

Furthermore, define $N=|\text{terms}(\alpha \oplus \beta)|$. Then, $\text{GASP}_\text{big}$ is the polynomial code $\text{PC}(K,L,T,N,\alpha,\beta)$.

**Definition 10:** Let $x,y \in \mathbb{Z}$ such that $x\leq y$. Then, the **set of all integers between $x$ and $y$** is defined by
\begin{align*}
    [x:y]=\{z: x\leq z \leq y \}.
\end{align*}

**Theorem 2:** The polynomial code $\text{GASP}_\text{big}$ is decodable and $T$-secure.

**Proof:** Initially, suppose that $L\leq K$. If $k \in [K]$ and $l \in [L]$, we have that
\begin{align*}
    \alpha_k+\beta_l=k-1+K(l-1).
\end{align*}

Since $k$ and $l$ can assume all values of, respectively, $[K]$ and $[L]$, it follows that each pair $(k,l)$ results in a unique integer in $[0:KL-1] \subset \mathbb{N}$. In addition, as every other term of $\alpha \oplus \beta$ is in $[KL:2KL+2T-2]$, we conclude that $\alpha \oplus \beta$ is decodable.

To prove that $\alpha \oplus \beta$ is $T$-secure, we observe that all $\alpha_{K+t}$ are distinct and all $\beta_{L+t}$ are distinct because $t \in [T]$, i.e., $\alpha_{K+t}$ and $\beta_{L+t}$ assume each of the $T$ elements in $[KL: KL+T-1] \subset \mathbb{N}$ based on the value of $t \in [T]$.

Now, suppose that $L>K$. If $k \in [K]$ and $l \in [L]$, we have that
\begin{align*}
    \alpha_l+\beta_k=K(l-1)+k-1.
\end{align*}

Similarly to the first case, since $k$ and $l$ can assume all values of, respectively, $[K]$ and $[L]$, it follows that each pair $(k,l)$ results in a unique integer in $[0:KL-1] \subset \mathbb{N}$ and every other term of $\alpha \oplus \beta$ is in $[KL:2KL+2T-2]$. So, we conclude that $\alpha \oplus \beta$ is decodable.

To prove that $\alpha \oplus \beta$ is $T$-secure, we observe that all $\alpha_{L+t}$ are distinct and all $\beta_{K+t}$ are distinct because $t \in [T]$.

Finally, by **Theorem 1**, it follows that the polynomial code $\text{GASP}_\text{big}$ is decodable and $T$-secure.

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Theorem 3:** Let $N=|\text{terms}(\alpha \oplus \beta)|$ where $\alpha$ and $\beta$ are as in **Definition 9**. Then, $N$ is given by

1. If $L\leq K$:
\begin{align*}
    N=
    \begin{cases}
        (K+T)(L+1)-1, & \text{ if }T<K.\\
        2KL+2T-1, & \text{ if }T\geq K.
    \end{cases}
\end{align*}

2. If $L> K$:
\begin{align*}
    N=
    \begin{cases}
        (L+T)(K+1)-1, & \text{ if }T<L.\\
        2KL+2T-1, & \text{ if }T\geq L.
    \end{cases}
\end{align*}

Consequently, by **Definition 4**, the polynomial $\text{GASP}_\text{big}(K,L,T)$ has a download rate 
\begin{align*}
    \mathcal{R}=\frac{KL}{N}, 
\end{align*}
where $N$ is as above.

**Proof:** Initially, suppose that $L\leq K$. The idea is to compute the number of terms in each of the four regions UL, UR, LL and LR, and use the inclusion-exclusion principle to obtain the number of terms in the whole table $\alpha \oplus \beta$. So,
\begin{align*}
    &\text{terms}(\text{UL})=[0:KL-1]\\
    &\text{terms}(\text{UR})=[KL:KL+K+T-2]\\
    &\text{terms}(\text{LL})=\bigcup_{l=1}^{L}[KL+K(l-1):KL+K(l-1)+T-1]\\
    &\text{terms}(\text{LR})=[2KL:2KL+2T-2]
\end{align*}

Thus, the cardinality of the above sets is as follows:
\begin{align*}
    &|\text{terms}(\text{UL})|=KL-1+1=KL\\
    &|\text{terms}(\text{UR})|=KL+T-2-KL+1=K+T-1\\
    &|\text{terms}(\text{LR})|=2KL+2T-2-2KL+1=2T-1
\end{align*}

To determine $|\text{terms}(\text{LL})|$, note that
\begin{align*}
    \text{LL}=
    \begin{bmatrix}
        KL & KL + K & \cdots & 2KL-2K & 2KL-K\\
        KL+1 & KL+K+1 & \cdots & 2KL-2K+1 & 2KL-K+1 \\
        \vdots & \vdots & \ddots & \vdots & \vdots\\
        KL+T-1 & KL + K+T-1 & \cdots & 2KL-2K+T-1 & 2KL-K+T-1
    \end{bmatrix}
    .
\end{align*}

When $T<K$, we have each entry of LL distinct. So, as LL has $T$ rows and $L$ columns, we have that
\begin{align*}
    |\text{terms}(\text{LL})|= LT.
\end{align*}

However, when $T\geq K$, note that the last $T-K$ entries of column $j$ will be repeated in the first $T-K$ entries of the column $j+1$. Thus, subtracting the repeated terms of the $LT$ entries,
\begin{align*}
    |\text{terms}(\text{LL})|&= TL-(L-1)(T-K)\\
    &=TL-TL+KL-K+T\\
    &=KL-K+T.
\end{align*}
Then, as the case $K=T$ is included when $T<K$, it follows that
\begin{align*}
    |\text{terms}(\text{LL})|=
    \begin{cases}
        LT, & \text{ if }T\leq K.\\
        KL-K+T, & \text{ if }T\geq K.
    \end{cases}
\end{align*}

Now, we compute the pairwise intersections. Since the largest term of UL is smaller than any term on the other blocks UR, LL and LR, the intersection between the set $\text{terms}(\text{UL})$ is disjoint from the term sets of the UR, LL and LR blocks. First, we compute $\text{terms}(\text{UR})\cap \text{terms}(\text{LL})$.

If $L=1$, LL is a matrix with one column with the terms, by (17), in $[K:K+T-1]$ which is repeated in UR, because $[K:K+T-1] \subset \text{terms}(\text{UR})=[K:K+T-1+(K-1)]$ by (16). Thus, 
\begin{align*}
    \text{terms}(\text{UR})\cap \text{terms}(\text{LL})=[K:K+T-1].
\end{align*}
If $L\geq 2$, UR contains the terms in the first column of LL, i.e. $[KL:KL+T-1]$, because $\text{terms}(\text{UR})=[KL:KL+T-1+(K-1)]$ by (16). But note that, by (18), LL has the terms between $[KL+K:KL+K+T-2]$ which is common with $UR$. In this way,
\begin{align*}
    \text{terms}(\text{UR})\cap \text{terms}(\text{LL})=[KL:KL+T-1] \cup [KL+K:KL+K+T-2].
\end{align*}

To compute $\text{terms}(\text{LL})\cap \text{terms}(\text{LR})$ and $\text{terms}(\text{UR})\cap \text{terms}(\text{LR})$, it is enough to use the definitions (16), (17) and (18) to determine the intersections without worrying about any specific case. In this way,  
\begin{align*}
    &\text{terms}(\text{LL})\cap \text{terms}(\text{LR})=[2KL:2KL-K+T-1],\\
    &\text{terms}(\text{UR})\cap \text{terms}(\text{LR})=[2KL:KL+K+T-2].
\end{align*}
    
Therefore, we have that
\begin{align*}
    &\text{terms}(\text{UR})\cap \text{terms}(\text{LL})=
    \begin{cases}
        [K:K+T-1], &\text{ if }L=1.\\
        [KL:KL+T-1] \cup [KL+K:KL+K+T-2], &\text{ if }L\geq 2.
    \end{cases}
    ,\\
    &\text{terms}(\text{LL})\cap \text{terms}(\text{LR})=[2KL:2KL-K+T-1],\\
    &\text{terms}(\text{UR})\cap \text{terms}(\text{LR})=[2KL:KL+K+T-2].
\end{align*}

Now, let us compute the cardinality of the three sets above. Initially, if $L=1$, using the previous definitions, we have
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})|=(T-1)+1=T.
\end{align*}
Remember that $|\text{terms}(\text{LL})|$ depends in the conditions $T\leq K $ and $T\geq K$. So, we check each case separately. If $L\geq 2$ and $T\leq K$, each term of LL are distinct and we do not need to exclude the repeated terms in the definition, i.e.,
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})|&=(T-1)+1+(T-2)+1\\
    &=2T-1.
\end{align*}
But, if $L\geq 2$ and $T\geq K$, as the last $T-K$ entries of column $j$ will be repeated in the first $T-K$ entries of the column $j+1$ in the block LL, it follows that
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})|&=(2T-1)-(T-K)\\
    &=K+T-1.
\end{align*}

Again, to compute $\text{terms}(\text{LL})\cap \text{terms}(\text{LR})$, we consider the cases $T\leq K $ and $T\geq K$. If $T\leq K$, we have $2KL>2KL-K+T-1$. So, 
\begin{align*}
    |\text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=0.
\end{align*}
On the other hand, if $T \geq K$, we have $2KL \leq 2KL-K+T-1$ and 
\begin{align*}
    |\text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=T-K.
\end{align*}

To compute $|\text{terms}(\text{UR})\cap \text{terms}(\text{LR})|$, we observe that, if $T\leq K(L-1)+1$, 
\begin{align*}
    2KL>2KL-1\geq KL+K+T-2
\end{align*}
and
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LR})|=0.
\end{align*}

Moreover, if $T\geq K(L-1)+1$, $2KL\leq KL+K+T-2$ and
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LR})|=T-(K(L-1)+1).
\end{align*}

In summary, 
\begin{align*}
    &|\text{terms}(\text{UR})\cap \text{terms}(\text{LL})|=
    \begin{cases}
        T, &\text{ if }L=1.\\
        2T-1, &\text{ if }L\geq 2, T\leq K.\\
        K+T-1, &\text{ if }L\geq 2, T \geq K.\\
    \end{cases}
    ,\\
    &|\text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=
    \begin{cases}
        0, &\text{ if } T\leq K.\\
        T-K, &\text{ if } T \geq K.
    \end{cases}
    ,\\
    &|\text{terms}(\text{UR})\cap \text{terms}(\text{LR})|=
    \begin{cases}
        0, &\text{ if } T\leq K(L-1)+1.\\
        T-(K(L-1)+1), &\text{ if } T \geq K(L-1)+1.
    \end{cases}
    .
\end{align*}

Finally, let us compute the triple intersection $\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})$. Note that $\text{terms}(\text{LL})\cap \text{terms}(\text{LR})$ and $\text{terms}(\text{UR})\cap \text{terms}(\text{LR})$ has the the smaller number $2KL>KL$ and the highest number, respectively, $2KL-K+T-1$ and $KL+K+T-2$ which are larger than $K+T-1$. Then, by intersection operation,
\begin{align*}
    \text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})=[2KL:\text{min}\{2KL-K+T-1,KL+K+T-2\}].
\end{align*}

Again, to compute the cardinality, we analyse each case.
    
If $L=1$, the inequality $K+T-1\leq 2K+T-2$ are satisfied. So, we have
\begin{align*}
    \text{min}\{K+T-1,2K+T-2\}=K+T-1.
\end{align*}
But, if $L=1$ and $T\leq K$, $2K>K+T-1.$ Thus,
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=0.
\end{align*}
On the other hand, if $L=1$ and $T>K$, we have that $2K\leq K+T-1$. Then,
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|&=K+T-1-2K+1\\
    &=T-K.
\end{align*}

Now, if $L\geq 2$, the inequality $KL+K+T-2\leq 2KL-K+T-1$ are satisfied. So, we have
\begin{align*}
    \text{min}\{2KL-K+T-1,KL+K+T-2\}=KL+K+T-2.
\end{align*}

In this way, if $L\geq 2$ and $T\leq K(L-1)+1$, $KL+K+T-2\leq 2KL-1$ which implies that $KL+K+T-2< 2KL$. Thus, by definition (22),
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=0.
\end{align*}

Furthermore, if $L\geq 2$ and $T> K(L-1)+1$, $KL+K+T-2\geq 2KL$. Thus,
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|&=KL+K+T-2-2KL+1\\
    &=T-(K(L-1)+1).
\end{align*}

In summary, for the triple intersection, we have the following results
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=
    \begin{cases}
        0, &\text{if }L=1, T\leq K.\\
        T-K, &\text{if }L= 1, T\geq K.\\
        0, &\text{if }L\geq 2, T \leq K(L-1)+1.\\
        T-(K(L-1)+1), &\text{if }L\geq 2, T \geq K(L-1)+1.
    \end{cases}
\end{align*}

We can now compute $N=|\text{terms}(\alpha \oplus \beta)|$ by using the inclusion-exclusion principle as follows
\begin{align*}
    N&=|\text{terms}(\alpha \oplus \beta)|=|\text{terms}(\text{UL})|+|\text{terms}(\text{UR})|+|\text{terms}(\text{LL})|+|\text{terms}(\text{LR})|\\
    &-|\text{terms}(\text{UR})\cap \text{terms}(\text{LL})-|\text{terms}(\text{LL})\cap \text{terms}(\text{LR})|-| \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|\\
    &+|\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|.
\end{align*}

Therefore, using all computed results together for $L\leq K$, we can conclude that
\begin{align*}
    N=
    \begin{cases}
        (K+T)(L+1)-1, & \text{ if }T<K.\\
        2KL+2T-1, & \text{ if }T\geq K.
    \end{cases}
\end{align*}

For $L>K$, the demonstration is similarly if we exchange $\alpha$ for $\beta$ and vice-versa, which implies that 
\begin{align*}
    N=
    \begin{cases}
        (L+T)(K+1)-1, & \text{ if }T<L.\\
        2KL+2T-1, & \text{ if }T\geq L.
    \end{cases}
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Theorem 4:** Let $N$ be defined as in **Theorem 3**. Then, if $L\leq K$,
\begin{align*}
    N \leq (K+T)(L+1)-1
\end{align*}
and, if $L>K$, 
\begin{align*}
    N \leq (L+T)(K+1)-1.
\end{align*}

**Proof:** We will start the demonstration by considering the case $L\leq K$. In this case, if $T<K$, 
\begin{align*}
    N=(K+T)(L-1)-1
\end{align*}
which satisfies the inequality $N \leq (K+T)(L+1)-1$.

If $L\leq K$ and $T\geq K$, we have $TL\geq KL$. Thus,
\begin{align*}
    N=2KL+2T-1=KL+T+KL+T-1\leq KL+K+TL+T-1.
\end{align*}

Now, if $L>K$, we simply do the same arguments by replacing $K$ with $L$ and vice versa to get that
\begin{align*}
    N \leq (L+T)(K+1)-1.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Finally, let us present a possible implementation/simulation of the process of using the polynomial code $\text{GASP}_\text{big}$. The methodology for presenting the code will be through an example where $K=L=3$, $T=2$, $p=29$, $A \in \text{GF}^{6 \times 2}(29)$ and $B \in \text{GF}^{2 \times 6}(29)$ defined as below:

In [49]:
A = GF29.Random((6,2))
B = GF29.Random((2,6))
print('A is')
print(A)
print('B is')
print(B)

A is
[[ 0  9]
 [17 21]
 [10 18]
 [ 0 21]
 [26  5]
 [ 2 19]]
B is
[[12 25 16 16  8 16]
 [23 11 14 14 28  8]]


First, we define a function that will calculate the number of servers we will need to apply the scheme in accordance with **Theorem 3**.

In [50]:
def number_of_servers_GASP_big(K,L,T):
    if L<=K:
        if T<K:
            N = (K+T)*(L+1)-1
        elif T>=K:
            N = 2*K*L+2*T-1
    elif L>K:
        if T<L:
            N = (L+T)*(K+1)-1
        elif T>=L:
            N = 2*K*L+2*T-1
    return N

In our example, we can check that we need $N=19$ servers by **Theorem 3** or by running the previous function as follows:

In [51]:
Numb_serv = number_of_servers_GASP_big(3, 3, 2)
print(Numb_serv)

19


Next, we define a function that will provide us with the $f(i)$ and $g(i)$ evaluations that each server will receive, stored in a dictionary with the labels $i \in [19]$. We show the full code below to explain the main parts (1) to (4) of the code.

In [52]:
def generate_shares_GASP_big(A, B, N, K, L, T, p, evaluation_vector):
    GF = galois.GF(p)

    # (1) Partition process
    A_submatrices = {}
    B_submatrices = {}
    m, n = A.shape
    l = B.shape[1]
    
    for i in range(K):
        submatriz = A[i * m // K :(i + 1) * m // K, :]
        A_submatrices[f"A_{i+1}"] = submatriz

    for i in range(L):
        submatriz = B[:, i * l // L:(i + 1) * l // L]
        B_submatrices[f"B_{i+1}"] = submatriz

    # (2) Process of choosing uniform random variables R_t and S_t at random
    R_submatrices = {}
    S_submatrices = {}
    for i in range(T):
        R_submatrices[f"R_{i+1}"] = GF.Random((m // K, n))
        S_submatrices[f"S_{i+1}"] = GF.Random((n, l // L))

    # (3) Definition of the degrees 
    alpha= [] 
    beta = []

    if L<=K:
        for i in range(K):
            alpha.append(i)
        for i in range(1, T+1):
            alpha.append(K*L + i - 1)
        for i in range(1, L+1):
            beta.append(K * (i-1))
        for i in range(1, T+1):
            beta.append(K*L + i - 1)
    elif L>K:
        for i in range(1, K+1):
            alpha.append(L * (i-1))
        for i in range(1, T+1):
            alpha.append(K*L + i - 1)
        for i in range(L):
            beta.append(i)
        for i in range(1, T+1):
            beta.append(K*L + i - 1)

    # (4) Definition of the evaluations
    f_evaluations = {}
    g_evaluations = {}
    for i in range(1,N+1):
        f_i = GF(np.zeros((m // K, n), dtype=int))
        for j in range(K):
            f_i += A_submatrices[f"A_{j+1}"] * GF(evaluation_vector[i-1])**(alpha[j]) 
        for j in range(T):
            f_i += R_submatrices[f"R_{j+1}"] * GF(evaluation_vector[i-1])**(alpha[K+j])

        g_i = GF(np.zeros((n, l // L), dtype=int))
        for j in range(L):
            g_i += B_submatrices[f"B_{j+1}"] * GF(evaluation_vector[i-1])**(beta[j])
        for j in range(T):
            g_i += S_submatrices[f"S_{j+1}"] * GF(evaluation_vector[i-1])**(beta[L+j])
        
        f_evaluations[i], g_evaluations[i] = f_i, g_i

    return f_evaluations, g_evaluations, alpha, beta

In (1), we partition the matrices $A$ and $B$ according to $K$ and $L$ to store the submatrices $A_k$ and $B_l$ in dictionaries “A_submatrices” and “B_submatrices”. Similarly, in (2), we define the random matrix variables $R_k$ and $S_l$ to keep the information in $A$ and $B$ safe. In (3), we define the vectors $\alpha$ and $\beta$ which will have the degrees to define the polynomials $f(x)$ and $g(x)$, respectively. The purpose of phases (1), (2) and (3) is to have enough information to define the polynomials $f(x)$ and $g(x)$ to create and send the evaluations through process (4) which uses an evaluation vector $\mathbf{a} \in \text{GF}^N(p)$ valid for the scheme. The function returns the set of evaluations of $f(x)$ and $g(x)$ with their labels, and the vectors $\alpha$ and $\beta$, as we will need these vectors to be input information for the function that decodes $AB$.

In our example, a valid evaluation vector would be $\mathbf{a}=[1,2,...,19]^t \in \text{GF}^{19}(29)$, because $\mathbf{a}=[1,2,...,19]^t$ satisfies the conditions of **Lemma 3**.

In [53]:
ev = GF29(np.array([i for i in range(1, Numb_serv+1)]))
print(ev)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


Thus, the set of evaluations of $f(x)$ and $g(x)$ can be obtained, respectively, as follows:

In [54]:
x, y, aph, bt = generate_shares_GASP_big(A, B, Numb_serv, 3, 3, 2, 29, ev)
print(x)
print(y)

{1: GF([[22, 22],
    [15, 15]], order=29), 2: GF([[24, 18],
    [13, 15]], order=29), 3: GF([[21,  5],
    [ 7, 25]], order=29), 4: GF([[21,  4],
    [20, 18]], order=29), 5: GF([[13, 19],
    [24,  0]], order=29), 6: GF([[22, 12],
    [22,  8]], order=29), 7: GF([[ 0,  1],
    [21, 24]], order=29), 8: GF([[23, 25],
    [22,  3]], order=29), 9: GF([[16, 21],
    [16, 19]], order=29), 10: GF([[14,  2],
    [13, 28]], order=29), 11: GF([[25, 13],
    [ 7,  2]], order=29), 12: GF([[20, 22],
    [27, 10]], order=29), 13: GF([[ 7,  6],
    [ 9, 22]], order=29), 14: GF([[11, 24],
    [14, 19]], order=29), 15: GF([[17, 22],
    [17, 11]], order=29), 16: GF([[11, 17],
    [14, 27]], order=29), 17: GF([[25,  3],
    [10, 28]], order=29), 18: GF([[15,  0],
    [ 9,  4]], order=29), 19: GF([[22, 15],
    [ 2, 14]], order=29)}
{1: GF([[20,  0],
    [ 1, 19]], order=29), 2: GF([[28, 28],
    [20, 14]], order=29), 3: GF([[ 5, 13],
    [18, 23]], order=29), 4: GF([[15, 22],
    [17,  5]], order=29),

Now, we define a function that generates the answers $h(i)=f(i)\cdot g(i)$ guided by the labels $i \in [N]$ that we put in the evaluations:

In [55]:
def generate_answers(x, y, N):
    h_evaluations = {}
    for i in range(1, N+1):
        h_evaluations[i] = x[i] @ y[i]

    return h_evaluations

Thus, we can verify the answers that the servers compute as follows:

In [56]:
z = generate_answers(x, y, Numb_serv)
print(z)

{1: GF([[27, 12],
    [25, 24]], order=29), 2: GF([[17, 25],
    [26, 23]], order=29), 3: GF([[21, 11],
    [21, 28]], order=29), 4: GF([[ 6, 18],
    [26,  8]], order=29), 5: GF([[24, 12],
    [ 4, 15]], order=29), 6: GF([[11,  7],
    [ 6,  0]], order=29), 7: GF([[9, 3],
    [0, 9]], order=29), 8: GF([[ 5, 20],
    [24,  8]], order=29), 9: GF([[ 6, 16],
    [ 0, 11]], order=29), 10: GF([[ 1, 13],
    [ 5, 16]], order=29), 11: GF([[22, 27],
    [ 0, 12]], order=29), 12: GF([[ 3,  5],
    [19, 10]], order=29), 13: GF([[16, 10],
    [ 2, 26]], order=29), 14: GF([[16,  1],
    [ 8, 18]], order=29), 15: GF([[ 0, 10],
    [28,  6]], order=29), 16: GF([[ 0,  4],
    [14, 14]], order=29), 17: GF([[ 5, 14],
    [22, 16]], order=29), 18: GF([[ 2,  4],
    [28,  3]], order=29), 19: GF([[17, 22],
    [16, 19]], order=29)}


Finally, we present the code to decode the multiplication $AB$. Similarly, we will show the code to explain the phases from (1) to (6).

In [57]:
def decodability_GASP(A, B, N, K, L, p, z, alpha, beta, evaluation_vector):
    GF = galois.GF(p)

    # (1) Definition of the degrees of h(x)=f(x)g(x)
    J = []
    for a in alpha:
        for b in beta:
            if a+b not in J:
                J.append(a+b)
    J = sorted(J)

    # (2) Start all submatrices A_kB_l as null matrices
    m = A.shape[0]
    l = B.shape[1]
    all_submatrices = {}
    ordered_pairs = []
    for k in range(1,K+1):
        for i in range(1,L+1):
            all_submatrices[f'A_{k}B_{i}'] = GF(np.zeros((m // K, l // L), dtype=int))
            ordered_pairs.append((k,i))

    # (3) Generalized Vandermonde matrix definition
    GV = GF(np.array([[GF(ai) ** j for j in J] for ai in evaluation_vector]))

    # (4) Decodabilty process
    for i in range(m // K):
        for j in range(l // L):
            y = []
            for m in range(1,N+1):
                y.append(z[m][i,j])
            y = GF(np.array(y))
            sol = np.linalg.solve(GV, y)
             
            f = 0
            for pair in ordered_pairs:
                # (5) Condition based on the distribution of degrees for the A_kB_l submatrices in the Degree Table
                if K>=L:
                    all_submatrices[f'A_{pair[1]}B_{pair[0]}'][i,j] = sol[f]
                    f += 1
                elif K<L:
                    all_submatrices[f'A_{pair[0]}B_{pair[1]}'][i,j] = sol[f]
                    f += 1

    # (6) Construction of the matrix AB
    AB = GF(np.block([[all_submatrices[f'A_{k}B_{i}'] for i in range(1, L + 1)] for k in range(1, K + 1)]))
                  
    return AB

In (1), we have the definition of all the degrees that appear in the polynomial $h(x)$. In (2), we start all the submatrices $A_kB_l$ that will make up the product $AB$ as null matrices to be updated in step (5). In (3), we define the Generalized Vandermonde matrix that defines the code $h(x)$. In (4) and (5), we solve the $\frac{m}{K} \times \frac{l}{L}$ systems to define the entries of the submatrices in a similar way to that shown in **Example 1**. Specifically in (5), we take care to check which degree accompanies each submatrix. As $K\geq L$, using the Degree Table, the terms appear in the following progression:
\begin{align*}
    h(x)=A_1B_1+A_2B_1x+A_3B_1x^2+...
\end{align*}

If $K<L$, the terms would appear as follows:
\begin{align*}
    h(x)=A_1B_1+A_1B_2x+A_1B_3x^2+...
\end{align*}

So we have to be careful because the list containing the degrees of $h(x)$ is in ascending order.

Finally, in (6), we compute the matrix $AB$ by assigning each block to the correct position. In our example, we would have that $AB$ is given by

In [58]:
decodability_GASP(A, B, Numb_serv, 3, 3, 29, z, aph, bt, ev)

GF([[ 4, 12, 10, 10, 20, 14],
    [20, 18, 15, 15, 28,  5],
    [12, 13,  6,  6,  4, 14],
    [19, 28,  4,  4,  8, 23],
    [21,  9, 22, 22,  0, 21],
    [26, 27,  8,  8, 26, 10]], order=29)

Since $A \in \text{GF}^{6 \times 2}(29)$ and $B \in \text{GF}^{2 \times 6}(29)$ have low dimensions in our example, we can check that the result is correct as follows:

In [59]:
print(A @ B)

[[ 4 12 10 10 20 14]
 [20 18 15 15 28  5]
 [12 13  6  6  4 14]
 [19 28  4  4  8 23]
 [21  9 22 22  0 21]
 [26 27  8  8 26 10]]


Since the download rate is always given by $\mathcal{R}=\frac{KL}{N}$, it follows that the download rate in this example is given by $\frac{9}{19}$.

## 8. Polynomial codes for small $T$

Now, we will present the family of polynomial codes $\text{GASP}_\text{small}$ which provides a higher download rate when $T<\text{min}\{K,L\}$ compared to $\text{GASP}_\text{big}$ (we will demonstrate this result after characterizing the value of $N$ for this case).

**Definition 11:** Let $K,L,T \in \mathbb{N}^*$. Let $\alpha$ and $\beta$ given by

1. If $L\leq K$:

\begin{align*}
\alpha_k=
\begin{cases}
    k-1,&\text{if }1\leq k \leq K\\
    KL+K(t-1),&\text{if }k=K+T, \ 1\leq t \leq T
\end{cases}
, \ \beta_l=
\begin{cases}
    K(l-1),&\text{if }1\leq l \leq L\\
    KL+t-1,&\text{if }l=L+T, \ 1\leq t \leq T
\end{cases}
.
\end{align*}

2. If $L> K$:

\begin{align*}
\alpha_l=
\begin{cases}
    K(l-1),& \text{if }1\leq l \leq L\\
    KL+t-1,& \text{if }l=L+T, \ 1\leq t \leq T
\end{cases}
, \ \beta_l=
\begin{cases}
    k-1,&\text{if }1\leq k \leq K\\
    KL+K(t-1),&\text{if }k=K+T, \ 1\leq t \leq T
\end{cases}
.
\end{align*}

Furthermore, define $N=|\text{terms}(\alpha \oplus \beta)|$. Then, $\text{GASP}_\text{small}$ is the polynomial code PC$(K,L,T,N,\alpha,\beta)$.

**Theorem 4:** The polynomial code $\text{GASP}_\text{small}$ is decodable and $T$-secure.

**Proof:** Initially, suppose that $L\leq K$. If $k \in [K]$ and $l \in [L]$, we have that
\begin{align*}
    \alpha_k+\beta_l=k-1+K(l-1).
\end{align*}

Since $k$ and $l$ can assume all values of, respectively, $[K]$ and $[L]$, it follows that each pair $(k,l)$ results in a unique integer in $[0:KL-1]$. In addition, as every other term of $\alpha \oplus \beta$ is in $[KL:2KL+KT-K+T-1]$, we conclude that $\alpha \oplus \beta$ is decodable.

To prove that $\alpha \oplus \beta$ is $T$-secure, we observe that all $\alpha_{K+t}$ are distinct and all $\beta_{L+t}$ are distinct because $t \in [T]$. I.e., $\alpha_{K+t}$ and $\beta_{L+t}$ assume each of the $T$ elements in, respectively, $[KL: KL +K(T-1)]$ and $[KL: KL+T-1]$ based on the value of $t \in [T]$.

Now, suppose that $L>K$. If $k \in [K]$ and $l \in [L]$, we have that
\begin{align*}
    \alpha_l+\beta_k=K(l-1)+k-1.
\end{align*}

Similarly to the first case, since $k$ and $l$ can assume all values of, respectively, $[K]$ and $[L]$, it follows that each pair $(k,l)$ results in a unique integer in $[0:KL-1]$ and every other term of $\alpha \oplus \beta$ is in $[KL:2KL+KT-K+T-1]$. So, we conclude that $\alpha \oplus \beta$ is decodable.

To prove that $\alpha \oplus \beta$ is $T$-secure, we observe that all $\alpha_{L+t}$ are distinct and all $\beta_{K+t}$ are distinct because $t \in [T]$.

Finally, by **Lemma 3**, it follows that the polynomial code $\text{GASP}_\text{small}$ is decodable and $T$-secure.   

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

The degree tables for $\text{GASP}_\text{small}$ when $K \leq L$ and $K>L$ are, respectively, as follows:

$$
\begin{aligned}
\begin{array}{|c|ccc|ccc|}
\hline
+                                                      & \beta_1=0                & \cdots                 & \beta_L=K(L-1)                & \beta_{L+1}=KL & \cdots & \beta_{L+T}=KL+T-1 \\ \hline
\alpha_1=0                                               & 0 & \cdots & K(L-1) & KL &  \cdots & KL+T-1 \\ 
\vdots                                                 & \vdots & \ddots & \vdots &   \vdots  &    \ddots  & \vdots \\ 
\alpha_K=K-1                                               & K-1 & \cdots & KL-1 & KL+K-1 &  \cdots  & KL+K+T-2 \\ \hline
\alpha_{K+1}=KL                   & KL &  \cdots  & 2KL-K & 2KL &  \cdots & 2KL+T-1 \\ 
\alpha_{K+2}=KL+K                   & KL+K &  \cdots  & 2KL & 2KL+K &  \cdots & 2KL+K+T-1 \\ 
\vdots                         & \vdots  & \ddots &  \vdots  & \vdots  &   \ddots  & \vdots \\ 
\alpha_{K+T}=KL+K(T-1) & KL+K(T-1) & \cdots & 2KL+K(T-2) & 2KL+K(T-1) & \cdots & 2KL+(K+1)(T-1) \\ \hline
\end{array}
\end{aligned}
$$


$$
\begin{aligned}
\begin{array}{|c|ccc|ccc|}
\hline
+                                                      & \beta_1=0                & \cdots                 & \beta_K=K-1                & \beta_{K+1}=KL & \cdots & \beta_{K+T}=KL+K(T-1) \\ \hline
\alpha_1=0                                               & 0 & \cdots & K-1 & KL &  \cdots  & KL+K(T-1) \\ 
\vdots                                                 & \vdots & \ddots & \vdots & \vdots &  \ddots  & \vdots \\ 
\alpha_L=K(L-1)                                               & K(L-1) & \cdots & KL-1 & 2KL-K & \cdots  & 2KL+K(T-2) \\ \hline
\alpha_{L+1}=KL                   & KL & \cdots & KL+K-1 & 2KL & \cdots & 2KL+K(T-1) \\ 
\alpha_{L+2}=KL+1                   & KL+1 & \cdots & KL+K & 2KL+1 & \cdots & 2KL+K(T-1)+1 \\ 
\vdots                         & \vdots & \ddots & \vdots  & \vdots  &   \ddots  & \vdots \\ 
\alpha_{L+T}=KL+T-1 & KL+T-1 & \cdots & KL+K+T-2 & 2KL+T-1 & \cdots & 2KL+(K+1)(T-1) \\ \hline
\end{array}
\end{aligned}
$$


Now, let us introduce a definition and a lemma to help us with the next theorem.

**Definition 12:** Let $x,y,z \in \mathbb{Z}$. Then, the set of all multiples of $z$ in the interval $[x:y]$ is defined by
\begin{align*}
    [x:y|z]=\{n \in \mathbb{Z}:x\leq n\leq y, n=cz,c \in \mathbb{Z}\}.
\end{align*}

**Lemma 8**: Let $x,y,z,w \in \mathbb{Z}$ such that $x=wz$. Then,
\begin{align*}
    |[x:y|z]|=\left( \left \lfloor{\frac{y}{z}}\right \rfloor-w+1 \right)^+
\end{align*}
where $c^+\doteq \text{max}\{0,c\}$, $\forall c \in \mathbb{Z}$.

**Proof:** Let $y=qz+r$ for $q \in \mathbb{Z}$ and $0 \leq r < z$. We analyse each case.

If $q=w$, since $0 \leq r < z$, there is no multiple of z between $wz$ and $wz+r$. So,
\begin{align*}
    |[wz:wz+r|z]|=0.
\end{align*}

If $q<w$, since $0 \leq r < z$, $x=wz<qz+r=y$. Thus, 
\begin{align*}
    |[wz:qz+r|z]|=0.
\end{align*}

If $q>w$, the smallest and largest multiples of z in $[x:y]$ are, respectively, $x=wz$ and $qz$. This implies that we have $q-w+1$ elements. Dividing $y=qz+r$ by $z$, observe that
\begin{align*}
    \frac{y}{z}=\frac{qz}{z}+\frac{r}{z}=q+\frac{r}{z}\implies \left \lfloor{\frac{y}{z}}\right \rfloor=q.
\end{align*}

Thus, 
\begin{align*}
    |[wz:qz+r|z]|&= q-w+1\\
    &=\left \lfloor{\frac{y}{z}}\right \rfloor - w +1.
\end{align*}
Putting all the above results together and consider the case $q<w$ which implies that $q-w+1\leq 0$, we can denote 
\begin{align*}
    |[x:y|z]|=\left( \left \lfloor{\frac{y}{z}}\right \rfloor-w+1 \right)^+.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Theorem 5**: Let $N=|\text{terms}(\alpha \oplus \beta)|$ where $\alpha$ and $\beta$ are as in **Definition 11**. Then, $N$ is given by

1. If $L\leq K$:
\begin{align*}
    N=
    \begin{cases}
        2K+T^2, & \text{if }L=1, T<K.\\
        KT+K+T, & \text{if }L=1,T\geq K.\\
        KL+K+L, & \text{if }L\geq 2,1=T<K.\\
        KL+K+L+T^2+T-3, & \text{if }L\geq 2,2\leq T<K.\\
        KL+KT+L+2T-3-\left \lfloor{\frac{T-2}{K}}\right \rfloor , & \text{if }L\geq 2,K\leq T\leq K(L-1)+1.\\
        2KL+KT-K+T, & \text{if }L\geq 2,K(L-1)+1\leq T.
    \end{cases}
\end{align*}

2. If $L> K$:
\begin{align*}
    N=
    \begin{cases}
        2L+T^2, & \text{if }K=1, T<L.\\
        LT+L+T, & \text{if }K=1,T\geq L.\\
        KL+K+L, & \text{if }K\geq 2,1=T<L.\\
        KL+K+L+T^2+T-3, & \text{if }K\geq 2,2\leq T<L.\\
        KL+LT+K+2T-3-\left \lfloor{\frac{T-2}{L}}\right \rfloor , & \text{if }K\geq 2,K\leq T\leq L(K-1)+1.\\
        2KL+LT-L+T, & \text{if }K\geq 2,L(K-1)+1\leq T.
    \end{cases}
\end{align*}

Consequently, the polynomial $\text{GASP}_\text{small}(K,L,T)$ has a download rate 
\begin{align*}
    \mathcal{R}=\frac{KL}{N}
\end{align*}
where $N$ is as above.

**Proof:** Initially, suppose that $L\leq K$. The idea is to compute the number of terms in each of the four regions UL, UR, LL and LR, and use the inclusion-exclusion principle to obtain the number of terms in the whole table $\alpha \oplus \beta$. So, by **Definition 11**,
\begin{align*}
    &\text{terms}(\text{UL})=[0:KL-1]\\
    &\text{terms}(\text{UR})=[KL:KL+K+T-2]\\
    &\text{terms}(\text{LL})=[KL:2KL+K(T-2)|K]\\
    &\text{terms}(\text{LR})=\bigcup_{t=1}^{T}[2KL+K(t-1):2KL+K(t-1)+T-1]
\end{align*}
Remember that in the block LL, $\alpha_k$ and $\beta_l$ are multiples of $K$, so $\alpha_k+\beta_l$ is a multiple of $K$, which justifies the definition above.

Thus, the cardinality of the terms of UL, UR and LL is as follows:
\begin{align*}
    &|\text{terms}(\text{UL})|=KL-1+1=KL\\
    &|\text{terms}(\text{UR})|=KL+K+T-2-KL+1=K+T-1\\
    &|\text{terms}(\text{LL})|=L+T-1
\end{align*}
where the value of $|\text{terms}(\text{LL})|$ follows from **Lemma 8**.

To compute $|\text{terms}(\text{LR})|$, we observe that when $T\leq K$, the matrix LR has all entries distinct. In addition, when $T\geq K$, the matrix LR has all entries with values from $2KL$ to $2KL+(K+1)(T-1)$ exactly. So,
\begin{align*}
    |\text{terms}(\text{LR})|=
    \begin{cases}
        T^2, & \text{ if }T< K.\\
        KT-K+T, & \text{ if }T\geq K.
    \end{cases}
\end{align*}

Now, we compute the pairwise intersections. Since the largest term of UL is smaller than any term on the other blocks UR, LL and LR, the intersection between the set $\text{terms}(\text{UL})$ is disjoint from the term sets of the UR, LL and LR blocks.

For the intersections of the of $\text{terms}(\text{UR})$ and $\text{terms}(\text{LR})$ with $\text{terms}(\text{LL})$, just remember that the terms in the LL block are all multiples of $K$. This implies that each term in these intersections is a multiple of $K$. Thus, we have 
\begin{align*}
    &\text{terms}(\text{UR}) \cap \text{terms}(\text{LL})=[KL:KL+K+T-2|K]\\
    &\text{terms}(\text{LL}) \cap \text{terms}(\text{LR})=[2KL:2KL+K(T-2)|K]
\end{align*}

To define $\text{terms}(\text{UR})\cap \text{terms}(\text{LR})$, note that, when $T\geq K$, we have $\text{terms}(\text{UR})=[KL:KL+K+T-2]$ and $\text{terms}(\text{LR})=[2KL:2KL+(K+1)(T-1)]$. So, 
\begin{align*}
    \text{terms}(\text{UR}) \cap \text{terms}(\text{LR})=[2KL:KL+K+T-2].
\end{align*}
This intersection is non-empty only when
\begin{align*}
    2KL\leq KL+K+T-2 \implies K(L-1)+2\leq T
\end{align*}
In this case, we have
\begin{align*}
    |\text{terms}(\text{UR}) \cap \text{terms}(\text{LR})|&=T-(K(L-1)+2)+1\\
    &=T-K(L-1)+1.
\end{align*}
So, for the general case $T\geq K$, we can consider the following result:
\begin{align*}
    |\text{terms}(\text{UR}) \cap \text{terms}(\text{LR})|=(T-K(L-1)+1)^+.
\end{align*}
On the other hand, if $T<K$, we have $2KL>KL+K+T-2$, except for the case $L=1$. In the case $L=1$, it follows that $\text{terms}(\text{UR}) \cap \text{terms}(\text{LR})=[2K:2K+T-2]$. Thus, we conclude that
\begin{align*}
    |\text{terms}(\text{UR}) \cap \text{terms}(\text{LR})|=
    \begin{cases}
        T-1, & \text{ if }T< K, L=1.\\
        0, & \text{ if }T< K, L\geq 2.\\
        (T-K(L-1)+1)^+, & \text{ if }T\geq K.
    \end{cases}
\end{align*}
Finally, let us compute the triple intersection $\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})$. Again, remember $\text{terms}(\text{LL})$ only has multiples of $K$. So, if $T \geq K$,  
\begin{align*}
    \text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})=[2KL:KL+K+T-2|K].
\end{align*}
In this case, by \textbf{Lemma 5.7}, it follows that
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=\left( \left \lfloor{\frac{T-2}{K}}\right \rfloor-L+2 \right)^+.
\end{align*}

Now, for the case $T<K$, we consider the two subcases $L=1$ and $L\geq 2$. If $L=1$ and $T=1$, we have that $2KL=2K>KL+K+T-2=2K-1$. So, the triple intersection are empty. However, if $L=1$ and $T\geq 2$, 
\begin{align*}
    2KL=2K\leq  KL+K+T-2=2K+T-2.
\end{align*}
In addition, note that 
\begin{align*}
    2K+T-2< 2K+K(T-2)
\end{align*}
when $T\geq 2$. Thus, $\text{terms}(\text{UR}) \cap \text{terms}(\text{LL})$ and $\text{terms}(\text{UR}) \cap \text{terms}(\text{LL})$, we have that
\begin{align*}
    \text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})=\{2K\}.
\end{align*}
Thus,
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=1.
\end{align*}

For the case $T<K$ and $L\geq 2$, we observe that $|\text{terms}(\text{UR})\cap \text{terms}(\text{LR})|=0$. So, 
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=0.
\end{align*} 

In summary,
\begin{align*}
    |\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|=
    \begin{cases}
        0, &\text{if }L=1, 1=T< K.\\
        1, &\text{if }L= 1, 2\leq T< K.\\
        0, &\text{if }L\geq 2, T < K.\\
        \left( \left \lfloor{\frac{T-2}{K}}\right \rfloor-L+2 \right)^+, &\text{if } T \geq K.
    \end{cases}
\end{align*}

We can now compute $N=|\text{terms}(\alpha \oplus \beta)|$ by using the inclusion-exclusion principle as follows
\begin{align*}
    N&=|\text{terms}(\alpha \oplus \beta)|=|\text{terms}(\text{UL})|+|\text{terms}(\text{UR})|+|\text{terms}(\text{LL})|+|\text{terms}(\text{LR})|\\
    &-|\text{terms}(\text{UR})\cap \text{terms}(\text{LL})-|\text{terms}(\text{LL})\cap \text{terms}(\text{LR})|-| \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|\\
    &+|\text{terms}(\text{UR})\cap \text{terms}(\text{LL})\cap \text{terms}(\text{LR})|.
\end{align*}

Therefore, using all computed results together for $L\leq K$, we can conclude that
\begin{align*}
    N=
    \begin{cases}
        2K+T^2, & \text{if }L=1, T<K.\\
        KT+K+T, & \text{if }L=1,T\geq K.\\
        KL+K+L, & \text{if }L\geq 2,1=T<K.\\
        KL+K+L+T^2+T-3, & \text{if }L\geq 2,2\leq T<K.\\
        KL+KT+L+2T-3-\left \lfloor{\frac{T-2}{K}}\right \rfloor , & \text{if }L\geq 2,K\leq T\leq K(L-1)+1.\\
        2KL+KT-K+T, & \text{if }L\geq 2,K(L-1)+1\leq T.
    \end{cases}
\end{align*}
The inequalities involving $T$ and $K(L-1)+1$ follow from the arguments used to conclude that $|\text{terms}(\text{UR}) \cap \text{terms}(\text{LR})|=(T-K(L-1)+1)^+$.
    
For $L>K$, the demonstration is similarly if we exchange $\alpha$ for $\beta$ and vice-versa.

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

**Theorem 6:** Let $T<\text{min}\{K,L\}$. In addition, let $N_\text{small}$ and $N_\text{big}$ be as, respectively, theorems **3** and **5**. Then, $N_\text{small}\leq N_\text{big}$.

**Proof:** If $L\leq K$ and $T=1<K$, we have that $\text{min}\{K,L\}=L$ and $T<L\leq K$. Thus, by the definitions of theorems **3** and **5**,
\begin{align*}
    N_\text{small}=KL+K+L=N_\text{big}.
\end{align*}
In addition, if $L\leq K$ and $T\geq 2$, we have $2\leq T<L\leq K$. So, the following inequality is satisfied
\begin{align*}
    0<1\leq LT-L-T^2+2,
\end{align*}
because the smallest values that $T$ and $L$ can take according to $2\leq T<L\leq K$ are $T=2$ and $L=3$, therefore, under the previous conditions, it follows
\begin{align*}
    \lim_{(T, L) \to (2, 3)} LT-L-T^2+2 = 1.
\end{align*}
But, 
\begin{align*}
    LT-L-T^2+2=(KL+K+LT+T-1)-(KL+K+L+T^2+T-3)=N_\text{big}-N_\text{small}
\end{align*}
Thus, we have
\begin{align*}
    N_\text{small}<N_\text{big}.
\end{align*}
Similarly, if $L> K$ and $T=1<L$, we have that $\text{min}\{K,L\}=K$ and $T<K<L$. Thus, by the definitions of theorems **3** and **5**,
\begin{align*}
    N_\text{small}=KL+K+L=N_\text{big}.
\end{align*}
Finally, if $L> K$ and $T\geq 2$, we have $2\leq T<K< L$. So, the following inequality is satisfied
\begin{align*}
    0<1\leq KT-K-T^2+2.
\end{align*}
because the smallest values that $T$ and $L$ can take according to $2\leq T<K\leq L$ are $T=2$ and $K=3$, therefore, under the previous conditions, it follows
\begin{align*}
    \lim_{(T, K) \to (2, 3)} KT-K-T^2+2 = 1.
\end{align*}
But, 
\begin{align*}
    KT-K-T^2+2=(KL+L+KT+T-1)-(KL+K+L+T^2+T-3)=N_\text{big}-N_\text{small}
\end{align*}
Thus, we have
\begin{align*}
    N_\text{small}<N_\text{big}.
\end{align*}

Therefore, for any $K,L, T\in \mathbb{N}^*$ such that $T<\text{min}\{K,L\}$, it follows that
\begin{align*}
    N_\text{small}\leq N_\text{big}.
\end{align*}

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \Box$

Finally, we present a possible implementation for $\text{GASP}_\text{small}$. Similarly, the idea is the same as that presented in the $\text{GASP}_\text{big}$. We will only modify the parts that refer to the definitions of the degrees $\alpha$ and $\beta$, and the number $N$ of servers, as presented in **Definition 11** and **Theorem 5**. In fact, we only need to create the functions that determine the number of servers and the $\alpha$ and $\beta$ vectors, since the functions for computing the responses and decodability the multiplication $AB$ work for both cases.

In [60]:
import math

def number_of_servers_GASP_small(K,L,T):
    if L<=K:
        if L==1 and T<K:
            N = 2*K + T**2
        elif L==1 and T>=K:
            N = K*T + K + T
        elif L>=2 and T==1 and T<K:
            N = K*L + K + L
        elif L>=2 and T>=2 and T<K:
            N = K*L + K + L + T**2 + T - 3
        elif L>=2 and K<=T<=K*(L-1)+1:
            N = K*L + K*T + L + 2*T - 3 - math.floor((T-2)/K)
        elif L>=2 and K*(L-1)+1<=T:
            N = 2*K*L + K*T - K + T
    elif L>K:
        if K==1 and T<L:
            N = 2*L + T**2
        elif K==1 and T>=L:
            N = L*T + L + T
        elif K>=2 and T==1 and T<L:
            N = K*L + K + L
        elif K>=2 and T>=2 and T<L:
            N = K*L + K + L + T**2 + T - 3
        elif K>=2 and L<=T<=L*(K-1)+1:
            N = K*L + L*T + K + 2*T - 3 - math.floor((T-2)/L)
        elif K>=2 and L*(K-1)+1<=T:
            N = 2*K*L + L*T - L + T
    return N

In [61]:
def generate_shares_GASP_small(A, B, N, K, L, T, p, evaluation_vector):
    GF = galois.GF(p)

    # (1) Partition process
    A_submatrices = {}
    B_submatrices = {}
    m, n = A.shape
    l = B.shape[1]
    
    for i in range(K):
        submatriz = A[i * m // K :(i + 1) * m // K, :]
        A_submatrices[f"A_{i+1}"] = submatriz

    for i in range(L):
        submatriz = B[:, i * l // L:(i + 1) * l // L]
        B_submatrices[f"B_{i+1}"] = submatriz

    # (2) Process of choosing uniform random variables R_t and S_t at random
    R_submatrices = {}
    S_submatrices = {}
    for i in range(T):
        R_submatrices[f"R_{i+1}"] = GF.Random((m // K, n))
        S_submatrices[f"S_{i+1}"] = GF.Random((n, l // L))

    # (3) Definition of the degrees 
    alpha= [] 
    beta = []

    if L<=K:
        for i in range(K):
            alpha.append(i)
        for i in range(1, T+1):
            alpha.append(K*L + K*(i-1))
        for i in range(1, L+1):
            beta.append(K * (i-1))
        for i in range(1, T+1):
            beta.append(K*L + i - 1)
    elif L>K:
        for i in range(1, K+1):
            alpha.append(L * (i-1))
        for i in range(1, T+1):
            alpha.append(K*L + i - 1)
        for i in range(L):
            beta.append(i)
        for i in range(1, T+1):
            beta.append(K*L + L*(i-1))

    # (4) Definition of the evaluations
    f_evaluations = {}
    g_evaluations = {}
    for i in range(1,N+1):
        f_i = GF(np.zeros((m // K, n), dtype=int))
        for j in range(K):
            f_i += A_submatrices[f"A_{j+1}"] * GF(evaluation_vector[i-1])**(alpha[j]) 
        for j in range(T):
            f_i += R_submatrices[f"R_{j+1}"] * GF(evaluation_vector[i-1])**(alpha[K+j])

        g_i = GF(np.zeros((n, l // L), dtype=int))
        for j in range(L):
            g_i += B_submatrices[f"B_{j+1}"] * GF(evaluation_vector[i-1])**(beta[j])
        for j in range(T):
            g_i += S_submatrices[f"S_{j+1}"] * GF(evaluation_vector[i-1])**(beta[L+j])
        
        f_evaluations[i], g_evaluations[i] = f_i, g_i

    return f_evaluations, g_evaluations, alpha, beta

Thus, we give an example with $K=L=3$, $T=2$, $p=29$ and the matrices $A \in \text{GF}^{6\times 2}(29)$ and $B \in \text{GF}^{6\times 2}(29)$ given the example of $\text{GASP}_{big}$, i.e., 

In [62]:
print('A is')
print(A)
print('B is')
print(B)

A is
[[ 0  9]
 [17 21]
 [10 18]
 [ 0 21]
 [26  5]
 [ 2 19]]
B is
[[12 25 16 16  8 16]
 [23 11 14 14 28  8]]


The number of servers in this case is given by $N=18$ which we can verify by **Theorem 5** or as follows:

In [63]:
Ns = number_of_servers_GASP_small(3, 3, 2)
print(Numb_serv)

19


Next, define a valid evaluation vector:

In [64]:
evl = GF29(np.array([i for i in range(1, Ns+1)]))
print(evl)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]


Thus, we can generate the set of evaluations of $f(x)$ and $g(x)$ as follows:

In [65]:
fl, gl, alpha, beta = generate_shares_GASP_small(A, B, Ns, 3, 3, 2, 29, evl)
print(fl)
print(fl)

{1: GF([[20,  7],
    [ 7, 11]], order=29), 2: GF([[ 6, 13],
    [21,  4]], order=29), 3: GF([[20, 13],
    [ 2, 11]], order=29), 4: GF([[ 9, 17],
    [ 4, 11]], order=29), 5: GF([[24, 11],
    [21, 16]], order=29), 6: GF([[ 3, 11],
    [ 1, 13]], order=29), 7: GF([[13, 21],
    [27, 13]], order=29), 8: GF([[16, 20],
    [ 0, 22]], order=29), 9: GF([[23,  9],
    [ 3,  9]], order=29), 10: GF([[ 3,  9],
    [12, 27]], order=29), 11: GF([[13,  0],
    [17, 17]], order=29), 12: GF([[ 0, 25],
    [28, 12]], order=29), 13: GF([[18, 24],
    [17, 17]], order=29), 14: GF([[ 4, 28],
    [ 1, 12]], order=29), 15: GF([[17, 28],
    [19, 27]], order=29), 16: GF([[24, 19],
    [18, 12]], order=29), 17: GF([[ 4, 14],
    [13,  6]], order=29), 18: GF([[27, 27],
    [ 0, 15]], order=29)}
{1: GF([[20,  7],
    [ 7, 11]], order=29), 2: GF([[ 6, 13],
    [21,  4]], order=29), 3: GF([[20, 13],
    [ 2, 11]], order=29), 4: GF([[ 9, 17],
    [ 4, 11]], order=29), 5: GF([[24, 11],
    [21, 16]], order=29), 

Now, we generate the set of answers $h(i)=f(i)\cdot g(i)$ that each server $i \in [18]$ computes:

In [66]:
hl = generate_answers(fl, gl, Ns)
print(hl)

{1: GF([[17,  5],
    [19, 24]], order=29), 2: GF([[14, 26],
    [ 1,  3]], order=29), 3: GF([[ 7, 12],
    [ 7,  6]], order=29), 4: GF([[ 3,  9],
    [11, 16]], order=29), 5: GF([[22,  5],
    [ 7, 27]], order=29), 6: GF([[ 0,  9],
    [24, 28]], order=29), 7: GF([[ 5, 11],
    [12, 13]], order=29), 8: GF([[6, 6],
    [1, 9]], order=29), 9: GF([[15, 26],
    [ 6, 18]], order=29), 10: GF([[24, 16],
    [21, 20]], order=29), 11: GF([[ 1, 23],
    [ 3, 24]], order=29), 12: GF([[ 1, 27],
    [20, 10]], order=29), 13: GF([[ 6, 16],
    [14, 27]], order=29), 14: GF([[25,  5],
    [10, 18]], order=29), 15: GF([[13, 17],
    [19,  7]], order=29), 16: GF([[27, 27],
    [ 1,  4]], order=29), 17: GF([[18, 27],
    [18, 12]], order=29), 18: GF([[ 6, 24],
    [ 1,  5]], order=29)}


Finally, we decode the same product $AB$ using the decode function:

In [67]:
ABsmall = decodability_GASP(A, B, Ns, 3, 3, 29, hl, alpha, beta, evl)
print(ABsmall)

[[ 4 12 10 10 20 14]
 [20 18 15 15 28  5]
 [12 13  6  6  4 14]
 [19 28  4  4  8 23]
 [21  9 22 22  0 21]
 [26 27  8  8 26 10]]


Again, we verify that the product $AB$ is correct as follows:

In [68]:
print(A @ B)

[[ 4 12 10 10 20 14]
 [20 18 15 15 28  5]
 [12 13  6  6  4 14]
 [19 28  4  4  8 23]
 [21  9 22 22  0 21]
 [26 27  8  8 26 10]]


We can observe that the download rate using the $\text{GASP}_{small}$ code is $\frac{9}{18}=\frac{1}{2}$ which is an improvement on the download rate of the $\text{GASP}_{big}$ code of $\frac{9}{19}$ applied to the same example. This observation follows from **Theorem 6**, since $T=2<\text{min}\{3,3\}=\text{min}\{K,L\}$.

## 9. Final considerations

We presented a possible solution to the SDMM problem using two types of $\text{GASP}$ codes, $\text{GASP}_\text{small}$ and $\text{GASP}_\text{big}$. In addition, we have seen that the study of the Decodability and $T$-Security properties can be facilitated by studying the Degree Table and that in certain situations the $\text{GASP}_\text{small}$ codes can be more efficient than the $\text{GASP}_\text{big}$ codes and vice versa.

Reference [3] presents a generalization for $\text{GASP}$ codes known by $\text{GASP}_r$ parametrized by an integer $r$ which outperforms all previously known polynomial codes for SDMM under an outer product partitioning. However, in [6], it is shown that these schemes can be improved by pre-computing the $R_kS_l$ submatrices that appear in the lower-right block of the degree table, which motivates the study of the SDMM problem with pre-computation. Therefore, if the reader is interested, we strongly recommend reading the references [3] and [6].

## References 

[1] W.-T. Chang and R. Tandon, “On the capacity of secure distributed matrix multiplication,” 2018 IEEE Global Communications Conference (GLOBECOM). IEEE, 2018.

[2] R. G. L. D’Oliveira, S. El Rouayheb and D. Karpuk, "GASP Codes for Secure Distributed Matrix Multiplication", in IEEE Transactions on Information Theory, 2020.

[3] R. G. L. D’Oliveira, S. E. Rouayheb, D. Heinlein and D. Karpuk, "Degree Tables for Secure Distributed Matrix Multiplication", in IEEE Journal on Selected Areas in Information Theory, 2021.

[4] R. G. L. D’Oliveira, S. E. Rouayheb, D. Heinlein, and D. Karpuk, “Notes on communication and computation in secure distributed matrix multiplication”, 2020 IEEE Conference on Communications and Network Security (CNS). IEEE, 2020.

[5] R. A. Machado, R. G. L. D’Oliveira, S. E. Rouayheb, and D. Heinlein, “Field trace polynomial codes for secure distributed matrix multiplication,” in 2021 XVII International Symposium "Problems of Redundancy in Information and Control Systems" (REDUNDANCY), 2021, pp. 188–193.

[6] R. Cartor, R. G. L. D'Oliveira, S. El Rouayheb, D. Heinlein, D. Karpuk and A. Sprintson, "Secure distributed matrix multiplication with precomputation"  2024 IEEE International Symposium on Information Theory (ISIT). IEEE, 2024.

[7] A. Shamir, “How to share a secret,” Commun. ACM, vol. 22, no. 11, pp. 612–613, 1979.

[8] G. R. Blakley, "Safeguarding Cryptographic Keys", Proceedings of the National Computer Conference, 48, 313-317, 1979.

[9] C. E. Shannon, “Communication theory of secrecy systems,” The Bell system technical journal, vol. 28, no. 4, pp. 656–715, 1949.

[10] I. S. Reed and G. Solomon, “Polynomial codes over certain finite fields,” Journal of the society for industrial and applied mathematics, vol. 8, no. 2, pp. 300–304, 1960.

[11] R. J. McEliece and D. V. Sarwate, “On sharing secrets and reed-solomon codes,” Communications of the ACM, vol. 24, no. 9, pp. 583–584, 1981.